In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# plot 한글출력
plt.rcParams['font.family'] = 'NanumGothic'

In [4]:
# warning message 제거
import warnings
warnings.filterwarnings('ignore')

In [5]:
staffs = pd.read_csv('../01.Data_Collection/staffs.csv')
writer_code = pd.read_csv('../01.Data_Collection/writer_code.csv')
writer_detail = pd.read_csv('../01.Data_Collection/writer_detail.csv')
boxoffice = pd.read_csv('../01.Data_Collection/boxoffice.csv')

print(staffs.shape)
print(writer_code.shape)
print(writer_detail.shape)
print(boxoffice.shape)

(107268, 4)
(1015, 5)
(12215, 6)
(802, 18)


In [5]:
print(staffs.isnull().sum())
staffs[staffs['peopleNm'] == '애덤 비첸']

movieCd        0
movieNm        0
peopleNm       0
staffRoleNm    0
dtype: int64


,movieCd,movieNm,peopleNm,staffRoleNm
15314,20100211,다이노 타임,애덤 비첸,시나리오(각본)


In [6]:
print(writer_code.isnull().sum())
writer_code[writer_code['repRoleNm'].isnull()]
# writer_code[writer_code['peopleNm'] == '애덤 비첸']
# writer_code['repRoleNm'].value_counts()

filmoNames    0
movieNm       0
peopleCd      0
peopleNm      0
repRoleNm     1
dtype: int64


,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
82,다이노 타임,다이노 타임,10045509,애덤 비첸,NaN


In [7]:
# 애덤 비첸 repRoleNm 채우기

writer_code.loc[writer_code['peopleNm'] == '애덤 비첸', 'repRoleNm'] = '시나리오(각본)'

writer_code.isnull().sum()

filmoNames    0
movieNm       0
peopleCd      0
peopleNm      0
repRoleNm     0
dtype: int64

# staffs - writer_code merge
movieNm peopleNm 으로 merge

In [8]:
# staffs 테이블에서 각본가만 따로 저장
# staffs 테이블에서 중복 제거
staff_writer_only = staffs[staffs['staffRoleNm'] == '시나리오(각본)'].drop_duplicates()

print(staff_writer_only.shape)
staff_writer_only.sample()

(961, 4)


,movieCd,movieNm,peopleNm,staffRoleNm
98287,20179230,도어락,박정희,시나리오(각본)


In [9]:
# writer_code 테이블에서 중복 제거
print(writer_code.shape)
writer_code = writer_code.drop_duplicates()

writer_list_count = pd.DataFrame(writer_code.groupby(['movieNm', 'peopleNm'])['peopleCd'].size())
print('2 번 이상 들어간 작가 수', len(writer_list_count[writer_list_count['peopleCd'] > 1]))
writer_list_count[writer_list_count['peopleCd'] > 1]
print(writer_code.shape)

(1015, 5)
2 번 이상 들어간 작가 수 0
(961, 5)


In [10]:
writer_movie_code_merge = staff_writer_only.merge(writer_code, how='inner', on=['movieNm', 'peopleNm'])

print(writer_movie_code_merge.shape)
print(writer_movie_code_merge.isnull().sum())
writer_movie_code_merge.sample()

(961, 7)
movieCd        0
movieNm        0
peopleNm       0
staffRoleNm    0
filmoNames     0
peopleCd       0
repRoleNm      0
dtype: int64


,movieCd,movieNm,peopleNm,staffRoleNm,filmoNames,peopleCd,repRoleNm
196,20156250,인천상륙작전,이재한,시나리오(각본),인천상륙작전 : 익스텐디드 에디션(확장판)|인천상륙작전|제3의 사랑|포화속으로|사요...,10057235,감독


## 위의 merge 한 테이블과 writer_detail 테이블 merge
movieCd, peopleCd

In [11]:
writer_movie_code_merge.sample()
writer_movie_code_merge.dtypes

movieCd         int64
movieNm        object
peopleNm       object
staffRoleNm    object
filmoNames     object
peopleCd        int64
repRoleNm      object
dtype: object

In [12]:
writer_detail.sample()
writer_detail.dtypes

movieCd        object
movieNm        object
moviePartNm    object
peopleCd        int64
peopleNm       object
sex            object
dtype: object

In [13]:
# writer_movie_code_merge 의 movieCd 컬럼이 int 이라서 str 로 바꿔준다

writer_movie_code_merge['movieCd'] = writer_movie_code_merge['movieCd'].apply(lambda x: str(x))
writer_movie_code_merge.dtypes

movieCd        object
movieNm        object
peopleNm       object
staffRoleNm    object
filmoNames     object
peopleCd        int64
repRoleNm      object
dtype: object

In [48]:
writers_all = writer_movie_code_merge.merge(writer_detail
                                            , how='inner'
                                            , left_on=['movieCd', 'peopleCd', 'staffRoleNm']
                                            , right_on=['movieCd', 'peopleCd', 'moviePartNm'])
writers_all = writers_all.drop_duplicates()
print(writers_all.shape)
writers_all.sample()

(945, 11)


,movieCd,movieNm_x,peopleNm_x,staffRoleNm,filmoNames,peopleCd,repRoleNm,movieNm_y,moviePartNm,peopleNm_y,sex
1391,20190470,이타미 준의 바다,정다운,시나리오(각본),이타미 준의 바다,20324792,감독,이타미 준의 바다,시나리오(각본),정다운,여자


In [49]:
len(writers_all['movieCd'].drop_duplicates())

670

In [50]:
# 다른 값 있는지 확인
# peopleNm 은 다른 값 있음, 활동명과 다른 이름인 것 같음, 일단 두개 다 남김

writers_all[writers_all['movieNm_x'] != writers_all['movieNm_y']]
writers_all[writers_all['peopleNm_x'] != writers_all['peopleNm_y']]

,movieCd,movieNm_x,peopleNm_x,staffRoleNm,filmoNames,peopleCd,repRoleNm,movieNm_y,moviePartNm,peopleNm_y,sex


In [51]:
# movieNm 은 x, y 내용 같으므로 컬럼 하나만 남긴다

writers_all = writers_all.drop('movieNm_y', axis=1)

In [52]:
# 컬럼명 정리
# peopleNm_x -> writerNm 필명
# peopleNm_y -> peopleNm 영화인 이름
writers_all = writers_all.rename(columns={'movieNm_x':'movieNm', 'peopleNm_x':'writerNm', 'peopleNm_y':'peopleNm'})

writers_all.sample()

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
183,20154323,슈퍼레이서 엔지,허선,시나리오(각본),극장판 뽀잉: 슈퍼 변신의 비밀|슈퍼레이서 엔지,20214806,감독,시나리오(각본),허선,NaN


## 전체 merge 한 파일 Null 값 확인

In [53]:
writers_all.isnull().sum()

movieCd         0
movieNm         0
writerNm        0
staffRoleNm     0
filmoNames      0
peopleCd        0
repRoleNm       0
moviePartNm     0
peopleNm        0
sex            89
dtype: int64

In [54]:
writers_all.sample()

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
236,20136000,레드 툼,구자환,시나리오(각본),해원|레드 툼|회색도시,10002241,감독,시나리오(각본),구자환,남자


## 영화코드별 행 하나로 합친다 
작가(전체 리스트)| 대표작가 | 작가성별 

In [75]:
# 작가 (전체 리스트)

movieCd_list = writers_all['movieCd'].unique()
# 영화코드를 바꿔준다

writer_list = []
for movieCd in movieCd_list:
    writers_of_movie = ''

    data = writers_all[writers_all['movieCd'] == movieCd]
    for writerNm in data['writerNm']:
        if writers_of_movie != '': writers_of_movie += ', '
        writers_of_movie += writerNm
        
    writer_list.append((movieCd, writers_of_movie))

writer_list

[('20148048', '류승완'),
 ('20148851', '이기철, 최동훈'),
 ('20149314', '우민호'),
 ('20145503', '장재현'),
 ('20148846', '조철현, 오승현, 이송원'),
 ('20148845', '이병헌'),
 ('20136068', '수오, 민지은'),
 ('20149570', '김수진, 이남규'),
 ('20148855', '한대덕, 곽경택'),
 ('20149569', '백운학'),
 ('20156557', '황혜순, 김정훈'),
 ('20157464', '김봉주'),
 ('20136888', '유하'),
 ('20140703', '이상현'),
 ('20143572', '김선정, 노경희, 박정예'),
 ('20148188', '김현석'),
 ('20142407', '박진표, 이병헌'),
 ('20149266', '한준희'),
 ('20142402', '이윤성, 민규동'),
 ('20145171', '박훈정'),
 ('20157465', '윤준형, 김영갑'),
 ('20142855', '이공주, 최관영'),
 ('20148461', '손용호, 안영진'),
 ('20145504', '노덕'),
 ('20156748', '서유민, 정기훈'),
 ('20143513', '김제영'),
 ('20131262', '하정우, 김주호'),
 ('20148841', '조사무엘, 신한솔'),
 ('20130566', '오승욱'),
 ('20136063', '김세희'),
 ('20148844', '김광태'),
 ('20148553', '최정미, 문제용, 유선동'),
 ('20140144', '김아영, 이종필'),
 ('20140193', '김익로, 천성일'),
 ('20143511', '최윤진'),
 ('20141224', '이해영'),
 ('20143512', '조중훈'),
 ('20157501', '하기호'),
 ('20136057', '손아람'),
 ('20130021', '박흥식, 최아름'),
 ('20155826'

In [79]:
writer_list = pd.DataFrame(writer_list, columns=['movieCd', '작가'])

print(writer_list.shape)
writer_list.sample()

(670, 2)


,movieCd,작가
402,20170590,김경찬


In [73]:
# 대표 작가 | 작가 코드 | 성별
first_writer = writers_all.groupby('movieCd')['peopleCd', 'writerNm', 'moviePartNm', 'sex'].first().reset_index()

print(first_writer.shape)
first_writer.sample()

(670, 5)


,movieCd,peopleCd,writerNm,moviePartNm,sex
346,20166132,20191566,이상호,시나리오(각본),남자


In [101]:
writer_info = writer_list.merge(first_writer, how='outer', on='movieCd')

# 배우 boxoffice merge 한 테이블 이용해서 merge

In [23]:
actor_boxoffice = pd.read_csv('../01.Data_Collection/films.csv')

print(actor_boxoffice.shape)
actor_boxoffice.sample()

(808, 24)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,...,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할,배우성별
652,55,런닝맨 : 풀룰루의 역습,2018.12.5,"1,420,218,750",0.2%,"1,420,218,750","186,912","186,912",511,"5,521",...,전체관람가,애니메이션,윤준상,"김서영,권창욱,엄상현,김연우,강시현,전태열",리우 목소리,20183745,10004461.0,김서영,배우,여자


In [29]:
actor_boxoffice['역할'].value_counts()

배우    778
감독      2
Name: 역할, dtype: int64

In [ ]:
# 영화코드로 merge

In [102]:
actor_boxoffice['영화코드'].dtypes
writer_info['movieCd'].dtypes
# dtype 다르다

writer_info['movieCd'] = writer_info['movieCd'].apply(lambda x: int(x))
writer_info['movieCd'].dtypes

dtype('int64')

In [104]:
writers_updated = actor_boxoffice.merge(writer_info, how='outer', left_on='영화코드', right_on='movieCd')

writers_updated = writers_updated.drop(['movieCd'], axis=1)
writers_updated = writers_updated.rename(columns={'peopleCd':'작가코드', 'writerNm': '대표작가', 'moviePartNm':'역할_시나리오(각본)', '역할':'역할_배우', 'sex':'작가성별'})
writers_updated.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,...,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
368,148.0,원스텝,2017.4.6,"13,002,000",0.0%,"13,002,000","1,662","1,662",33,239,...,20172863.0,20169728.0,박산다라,배우,여자,"홍승표, 박지현, 전재홍, 조성걸",20275129.0,홍승표,시나리오(각본),남자


In [105]:
writers_updated.isnull().sum()

순위               2
영화명              2
개봉일              2
매출액              2
매출액점유율           2
누적매출액            2
관객수              2
누적관객수            2
스크린수             2
상영횟수             2
대표국적             2
국적               2
제작사             22
배급사              3
등급               2
장르               2
감독               5
배우              33
배역명            451
영화코드             2
배우코드            30
주연배우            25
역할_배우           30
배우성별            25
작가             140
작가코드           140
대표작가           140
역할_시나리오(각본)    140
작가성별           174
dtype: int64

In [106]:
writers_updated[writers_updated['영화명'].isnull()]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,...,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,윤용진,20110475.0,윤용진,시나리오(각본),남자
809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,김의곤,20244869.0,김의곤,시나리오(각본),남자


In [107]:
writers_all[writers_all['writerNm'] == '윤용진']

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
1315,20183144,선종 무문관,윤용진,시나리오(각본),선종 무문관|할|내 마음 속에 작은 영웅들,20110475,감독,시나리오(각본),윤용진,남자


In [111]:
writers_updated['영화코드'] = writers_updated['영화코드'].fillna(20183144)
writers_updated['영화명'] = writers_updated['영화명'].fillna('선종 무문관')

writers_updated.isnull().sum()

순위               2
영화명              0
개봉일              2
매출액              2
매출액점유율           2
누적매출액            2
관객수              2
누적관객수            2
스크린수             2
상영횟수             2
대표국적             2
국적               2
제작사             22
배급사              3
등급               2
장르               2
감독               5
배우              33
배역명            451
영화코드             0
배우코드            30
주연배우            25
역할_배우           30
배우성별            25
작가             140
작가코드           140
대표작가           140
역할_시나리오(각본)    140
작가성별           174
dtype: int64

# merge file EDA

In [120]:
df = pd.read_csv('../01.Data_Collection/films.csv')

print(df.shape)
df.sample()

(810, 29)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
70,29.0,손님,2015.7.9,"6,385,937,880",0.7%,"6,385,937,880","828,025","828,025",634,"23,404",한국,한국,(주)유비유필름,씨제이이앤엠(주),15세이상관람가,"판타지,공포(호러)",김광태,"류승룡,이성민,천우희,이준,정경호,김정영,박시연,이민지,백승호,유재현,이서연,이승준...",NaN,20148844.0,10019065.0,류승룡,배우,남자,김광태,20147149.0,김광태,시나리오(각본),남자


In [121]:
df.isnull().sum()

순위               2
영화명              0
개봉일              2
매출액              2
매출액점유율           2
누적매출액            2
관객수              2
누적관객수            2
스크린수             2
상영횟수             2
대표국적             2
국적               2
제작사             22
배급사              3
등급               2
장르               2
감독               5
배우              33
배역명            451
영화코드             0
배우코드            30
주연배우            25
역할_배우           30
배우성별            25
작가             140
작가코드           140
대표작가           140
역할_시나리오(각본)    140
작가성별           174
dtype: int64

In [129]:
for i in df[df['작가'].isnull()]['영화코드']:
    if int(i) in writers_all['movieCd']: print(writers_all[writers_all['movieCd'] == i]['movieNm'])

In [132]:
df_full = df.drop('배역명', axis=1).dropna()

print(df_full.shape)
df_full.sample()

(606, 28)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
404,35.0,리얼,2017.6.28,"3,357,312,530",0.4%,"3,357,312,530","470,105","470,105",970,"25,139",한국,한국,(주)코브픽쳐스,씨제이이앤엠(주),청소년관람불가,액션,이사랑,"김수현,성동일,이성민,최진리,조우진,이경영,김홍파,정인겸,최권,한지은,문광무,박서준...",20159647.0,20140440.0,김수현,배우,남자,이정섭,10057277.0,이정섭,시나리오(각본),남자


In [135]:
df_full.groupby(['배우성별', '작가성별'])['영화코드'].size().reset_index()

,배우성별,작가성별,영화코드
0,남자,남자,327
1,남자,여자,85
2,여자,남자,131
3,여자,여자,63


# F-rated 컬럼 만들기


In [206]:
df = final_merge

f_rated = []
for i, row in df.iterrows():
    answer = 0
    if row['배우성별'] == '여자' : answer += 1
    if row['작가성별'] == '여자' : answer += 1
    if row['감독성별'] == '여자' : answer += 1
        
    f_rated.append(answer)
df['F-rated'] = f_rated
df.sample()

,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
396,1,신준,감독,남자,20188787.0,"용순|용순, 열 여덟 번째 여름|경쟁19 섹션5",2017-06-08,13268.0,한국,13268.0,97538200.0,신준,한국,신준,신준,15세이상관람가,97538200.0,0.0%,롯데쇼핑㈜롯데엔터테인먼트,용순,"이수경,최덕문,박근록,김동영,장햇살,최여진,김한나,박운아,박지연,유리,윤혜인,윤혜인...",배우,여자,20195643.0,기묘한 가족|침묵|용순|특별시민|차이나타운|방황하는 칼날|우아한 거짓말|오오단편극장...,2558.0,86.0,126.0,용순,20168041,신준,감독,남자,20188787.0,"용순|용순, 열 여덟 번째 여름|경쟁19 섹션5",드라마,(주)아토,이수경


In [207]:
# F-rated 까지 업데이트한 파일 저장
df.to_csv('../01.Data_Collection/films.csv', index=False)
pd.read_csv('../01.Data_Collection/films.csv')['F-rated'].value_counts()

0    423
1    271
2     70
3     46
Name: F-rated, dtype: int64

In [31]:
# 영화명 중복값 있는지 확인
# boxoffice 테이블에는 movieCd 없기 때문에 영화명으로 찾아야 한다
data = pd.DataFrame(boxoffice['영화명'].value_counts().reset_index())

data[data['영화명'] > 1]

,index,영화명


In [131]:
writers_boxoffice_merge = writers_all.merge(boxoffice, how='outer', left_on='movieNm', right_on='영화명')
writers_boxoffice_merge.sample()

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex,...,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
2927,20165562,소녀의 세계,안정민,시나리오(각본),소녀의 세계,20238968.0,감독,감독,안정민,남자,...,14,208,한국,한국,"(주)빅오픽쳐스,(주)날개엔터테인먼트",(주)드림팩트엔터테인먼트,12세이상관람가,멜로/로맨스,안정민,"노정의,조수향,권나라,조수하,김수현,김용연,김현주,박솔,박수연,이경훈"


In [132]:
writers_boxoffice_merge.isnull().sum()

movieCd        132
movieNm        132
writerNm       132
staffRoleNm    132
filmoNames     132
peopleCd       132
repRoleNm      132
moviePartNm    132
peopleNm       132
sex            298
순위              32
영화명             32
개봉일             32
매출액             32
매출액점유율          32
누적매출액           32
관객수             32
누적관객수           32
스크린수            32
상영횟수            32
대표국적            32
국적              32
제작사            146
배급사             35
등급              32
장르              32
감독              35
배우             131
dtype: int64

# boxoffice - 배우 - 작가 테이블과 감독 정보 테이블 merge 

In [300]:
data = pd.read_csv('../01.Data_Collection/director_merged.csv')
directors_all = data[data['moviePartNm'] == '감독']

print(directors_all.shape)
directors_all.sample()

(869, 9)


,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex,repRoleNm_x,filmoNames,repRoleNm_y
335,20158682,꾸루꾸루와 친구들: 무지개 나무의 비밀,감독,10026803,문제대,남자,감독,꾸루꾸루와 친구들: 무지개 나무의 비밀,감독


In [301]:
len(directors_all['movieCd'].unique())

788

In [302]:
len(df['영화코드'].unique())

809

In [309]:
# 대표 감독 | 감독 코드 | 성별
first_director = directors_all.groupby(['movieCd', 'movieNm'])['peopleCd', 'peopleNm', 'moviePartNm', 'sex'].first().reset_index()

print(first_director.shape)
first_director.sample()

(788, 6)


,movieCd,movieNm,peopleCd,peopleNm,moviePartNm,sex
421,20167903,원더풀 고스트,10066770,조원희,감독,남자


In [310]:
data = df.merge(first_director, how='outer', left_on=['영화코드', '영화명'], right_on=['movieCd', 'movieNm']).drop_duplicates()
# data = data.drop(['movieCd'], axis=1)
df_all = data.rename(columns={'peopleCd': '감독코드', 'peopleNm':'대표감독', 'moviePartNm':'역할_감독', 'sex':'감독성별'})

print(df_all.shape)
df_all.sample()

(839, 36)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,F-rated,movieCd,movieNm,감독코드,대표감독,역할_감독,감독성별
14,17.0,쎄시봉,2015.2.5,"13,571,515,911",1.5%,"13,571,515,911","1,715,370","1,715,370",829,"51,744",한국,한국,"유한회사 제이필름,(주)무브픽쳐스",씨제이이앤엠(주),15세이상관람가,"멜로/로맨스,코미디,드라마",김현석,"김윤석,정우,한효주,김희애,진구,장현성,강하늘,조복래,김인권,장미진,강명찬,정수교,...",NaN,20148188.0,10005276.0,김윤석,배우,남자,김현석,10006261.0,김현석,시나리오(각본),남자,0.0,20148188.0,쎄시봉,10006261.0,김현석,감독,남자


In [305]:
df_all[df_all['영화명'].isnull()]['movieCd'].value_counts()

20172243.0    1
20191029.0    1
20168962.0    1
20189862.0    1
20197441.0    1
20186707.0    1
20193828.0    1
Name: movieCd, dtype: int64

In [306]:
for i in df_all[df_all['영화명'].isnull()]['movieCd'].unique():
    print(directors_all[directors_all['movieCd'] == i]['movieNm'])

1054    입영전야 무삭제: 여선배의 자취방 버전
Name: movieNm, dtype: object
790    최악의 하루+폴라로이드 작동법
Name: movieNm, dtype: object
1959    오늘도 위위
Name: movieNm, dtype: object
612    감금의 시간 2 무삭제
Name: movieNm, dtype: object
2116    82년생 김지영
Name: movieNm, dtype: object
2030    북간도의 십자가
Name: movieNm, dtype: object
401    재혼의 기술
Name: movieNm, dtype: object


In [307]:
df[df['영화명'].str.contains('입영전야')]        # 없음
df[df['영화명'].str.contains('최악의 하루')]     # 없음
df[df['영화명'].str.contains('오늘도 위위')]     # 없음
df[df['영화명'].str.contains('감금의 시간')]     # 없음 (1, 2 다른 영화)
df[df['영화명'].str.contains('82년생 김지영')]   # 없음
df[df['영화명'].str.contains('북간도의 십자가')] # 없음
df[df['영화명'].str.contains('재혼의 기술')]     # 없음

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,F-rated


In [308]:
directors_all[directors_all['movieNm'].str.contains('입영전야')]
directors_all[directors_all['movieNm'].str.contains('최악의 하루')]
directors_all[directors_all['movieNm'].str.contains('오늘도 위위')]
directors_all[directors_all['movieNm'].str.contains('감금의 시간')]
directors_all[directors_all['movieNm'].str.contains('82년생 김지영')]
directors_all[directors_all['movieNm'].str.contains('북간도의 십자가')]
directors_all[directors_all['movieNm'].str.contains('재혼의 기술')]

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex,repRoleNm_x,filmoNames,repRoleNm_y
401,20197441,재혼의 기술,감독,10065915,조성규,남자,감독,재혼의 기술|발광하는 현대사|늦여름|딥|각자의 미식|게스트하우스|실종2|어떻게 헤어...,감독


In [316]:
# 영화명, 영화코드 없는 데이터 입력해줌
df_all['영화코드'] = df_all['영화코드'].fillna('movieCd')
df_all['영화명'] = df_all['영화명'].fillna('movieNm')

df_all.isnull().sum()
# movieCd 컬럼 삭제
df_all = df_all.drop(['movieCd', 'movieNm'], axis=1)
print(df_all.shape)
df_all.sample()

(839, 34)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,F-rated,감독코드,대표감독,역할_감독,감독성별
45,92.0,명량: 회오리 바다를 향하여,2015.5.7,"64,170,500",0.0%,"64,170,500","8,897","8,897",92,824,한국,한국,(주)빅스톤픽쳐스,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,다큐멘터리,"정세교,김한민","김한민,오타니 료헤이,이해영,권율,장준녕",NaN,2.01574e+07,10006204.0,김한민,배우,남자,NaN,NaN,NaN,NaN,NaN,0.0,20158266.0,정세교,감독,남자


In [317]:
# 최종 merge 파일 저장

df_all.to_csv('../01.Data_Collection/films.csv', index=False)

In [319]:
# 잘 저장되었는지 확인
df = pd.read_csv('../01.Data_Collection/films.csv')

print(df.shape)
df.sample()

(839, 34)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,F-rated,감독코드,대표감독,역할_감독,감독성별
349,68.0,레이디버그,2017.2.22,"310,291,500",0.0%,"310,291,500","41,041","41,041",172,"2,158",한국,"한국,프랑스,일본",(주)삼지애니메이션,(주)삼지애니메이션,전체관람가,"애니메이션,액션,어드벤처",장성,"여민정,남도형,홍시호,이민규",레이디버그,20168668.0,20234807.0,여민정,배우,여자,NaN,NaN,NaN,NaN,NaN,1.0,20275130.0,장성,감독,남자


# 영화별 개봉년도 컬럼 추가하기
boxoffice 테이블의 개봉일 컬럼 이용

In [32]:
# # 개봉일에서 앞 4자리만 가져와서 개봉년도로 넣어준다
# boxoffice['개봉년도'] = boxoffice['개봉일'].apply(lambda x: x[:4])
# boxoffice.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,개봉년도
781,5,나쁜 녀석들: 더 무비,2019.9.11,"39,541,671,284",5.0%,"39,541,671,284","4,571,217","4,571,217","1,511","113,798",한국,한국,"씨제이이앤엠(주),(주)영화사비단길",씨제이이앤엠(주),15세이상관람가,"범죄,액션",손용호,"마동석,김상중,김아중,장기용,강예원,김대영,박형수,김혜윤",2019


In [34]:
# # 영화명으로 writer_sex_ratio 테이블과 boxoffice['개봉년도']를 merge 해준다
# data = writers_all.merge(boxoffice[['영화명', '개봉년도']], how='left', left_on='movieNm', right_on='영화명')

# data = data.drop('영화명', axis=1)
# writers_all = data.rename(columns={'개봉년도':'year'})
# writers_all.sample()

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex,year
1935,20173903,역모-반란의 시대,김황성,시나리오(각본),역모-반란의 시대,20283090,시나리오(각본),시나리오(각본),김황성,NaN,2017


# [궁금한 것] movieCd 앞 4자리와 실제 개봉년도 관계 확인하기
movieCd 앞 4자리도 년도처럼 보이는데, 실제 개봉년도와는 차이나는 항목들이 있음

movieCd 앞 4자리는 영화 제작 시작년도? 영진위에 등록한 년도?

# Null 값 확인

In [35]:
writers_all.isnull().sum()

movieCd          0
movieNm          0
writerNm         0
staffRoleNm      0
filmoNames       0
peopleCd         0
repRoleNm        0
moviePartNm      0
peopleNm         0
sex            166
year            32
dtype: int64

In [36]:
writers_all[writers_all['sex'].isnull()]

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex,year
176,20156807,뽀로로 극장판 컴퓨터 왕국 대모험,이인균,시나리오(각본),뽀로로 극장판 컴퓨터 왕국 대모험|뽀로로 극장판 슈퍼썰매 대모험,10057072,시나리오(각본),시나리오(각본),이인균,NaN,2015
218,20100211,다이노 타임,애덤 비첸,시나리오(각본),다이노 타임,10045509,시나리오(각본),시나리오(각본),애덤 비첸,NaN,2015
246,20144442,조선마술사,조정화,시나리오(각본),조선마술사|슬기로운 해법|화성으로 간 사나이|해피버스데이|교회누나|묻지마 패밀리,10066948,시나리오(각본),시나리오(각본),조정화,NaN,2015
247,20144442,조선마술사,조정화,시나리오(각본),조선마술사|슬기로운 해법|화성으로 간 사나이|해피버스데이|교회누나|묻지마 패밀리,10066948,시나리오(각본),시나리오(각본),조정화,NaN,2015
248,20144442,조선마술사,조정화,시나리오(각본),조선마술사|슬기로운 해법|화성으로 간 사나이|해피버스데이|교회누나|묻지마 패밀리,10066948,시나리오(각본),시나리오(각본),조정화,NaN,2015
249,20144442,조선마술사,조정화,시나리오(각본),조선마술사|슬기로운 해법|화성으로 간 사나이|해피버스데이|교회누나|묻지마 패밀리,10066948,시나리오(각본),시나리오(각본),조정화,NaN,2015
344,20154323,슈퍼레이서 엔지,허선,시나리오(각본),극장판 뽀잉: 슈퍼 변신의 비밀|슈퍼레이서 엔지,20214806,감독,감독,허선,NaN,2015
345,20154323,슈퍼레이서 엔지,허선,시나리오(각본),극장판 뽀잉: 슈퍼 변신의 비밀|슈퍼레이서 엔지,20214806,감독,기획,허선,NaN,2015
346,20154323,슈퍼레이서 엔지,허선,시나리오(각본),극장판 뽀잉: 슈퍼 변신의 비밀|슈퍼레이서 엔지,20214806,감독,시나리오(각본),허선,NaN,2015
347,20154323,슈퍼레이서 엔지,허선,시나리오(각본),극장판 뽀잉: 슈퍼 변신의 비밀|슈퍼레이서 엔지,20214806,감독,프로듀서,허선,NaN,2015


# [할 것] sex 컬럼 Null 값 채우기

In [37]:
## null 값 검색해서 채우기...

# 영화 리스트 다른지 확인
boxoffice 기준으로?

In [66]:
writers_all['movieCd'].value_counts()

20140191    50
20186646    40
20156441    40
20179263    40
20165981    34
20175865    33
20180804    32
20166128    30
20153307    30
20175581    30
20143717    29
20176101    28
20188117    28
20180401    28
20179163    27
20164527    26
20198978    24
20159657    21
20155382    21
20179442    21
20169730    21
20174199    20
20148460    20
20162972    20
20174647    20
20193017    20
20181142    20
20155335    20
20161568    19
20168773    19
            ..
20178161     1
20172641     1
20162461     1
20140703     1
20180063     1
20150966     1
20142621     1
20158483     1
20153313     1
20151228     1
20158504     1
20152746     1
20186651     1
20156047     1
20167102     1
20190741     1
20153316     1
20141589     1
20141060     1
20166127     1
20152402     1
20159224     1
20167903     1
20162145     1
20170670     1
20174846     1
20198183     1
20160261     1
20179006     1
20166384     1
Name: movieCd, Length: 678, dtype: int64

In [74]:
writer_code.sample()
# ['movieCd'].value_counts()

,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
430,원스텝,원스텝,20275129,홍승표,시나리오(각본)


In [77]:
writer_movie_code_merge['movieCd'].value_counts()

20161568    8
20183745    5
20156441    5
20140704    5
20150979    5
20158242    5
20175527    4
20161603    4
20100211    4
20161762    4
20151227    4
20153801    4
20140241    4
20168161    4
20149258    4
20143717    4
20157064    4
20189746    4
20172863    4
20182490    3
20187322    3
20168250    3
20175581    3
20177845    3
20173965    3
20174348    3
20171817    3
20174243    3
20179205    3
20172469    3
           ..
20165045    1
20169730    1
20136869    1
20143753    1
20153313    1
20152402    1
20198323    1
20160041    1
20172642    1
20143511    1
20130260    1
20120111    1
20185784    1
20155744    1
20188393    1
20163041    1
20162442    1
20141060    1
20141224    1
20166421    1
20143510    1
20163521    1
20153443    1
20176122    1
20160522    1
20172204    1
20181561    1
20145504    1
20157464    1
20166384    1
Name: movieCd, Length: 679, dtype: int64

In [81]:
writer_movie_code_merge[writer_movie_code_merge['movieNm'] == '아빠는 딸']

,movieCd,movieNm,peopleNm,staffRoleNm,filmoNames,peopleCd,repRoleNm
430,20151227,아빠는 딸,김지선,시나리오(각본),아빠는 딸|올드미스다이어리,20111544,시나리오(각본)
431,20151227,아빠는 딸,진나리,시나리오(각본),아빠는 딸|심장이 뛰네,20145369,시나리오(각본)
432,20151227,아빠는 딸,최윤미,시나리오(각본),아빠는 딸,20289451,시나리오(각본)
433,20151227,아빠는 딸,조성우,시나리오(각본),아빠는 딸,20288153,시나리오(각본)


In [82]:
writer_movie_code_merge['movieCd'].value_counts()

20161568    8
20183745    5
20156441    5
20140704    5
20150979    5
20158242    5
20175527    4
20161603    4
20100211    4
20161762    4
20151227    4
20153801    4
20140241    4
20168161    4
20149258    4
20143717    4
20157064    4
20189746    4
20172863    4
20182490    3
20187322    3
20168250    3
20175581    3
20177845    3
20173965    3
20174348    3
20171817    3
20174243    3
20179205    3
20172469    3
           ..
20165045    1
20169730    1
20136869    1
20143753    1
20153313    1
20152402    1
20198323    1
20160041    1
20172642    1
20143511    1
20130260    1
20120111    1
20185784    1
20155744    1
20188393    1
20163041    1
20162442    1
20141060    1
20141224    1
20166421    1
20143510    1
20163521    1
20153443    1
20176122    1
20160522    1
20172204    1
20181561    1
20145504    1
20157464    1
20166384    1
Name: movieCd, Length: 679, dtype: int64

In [68]:
writer_detail['movieCd'].value_counts()

20140191    54
20156441    41
20186646    40
20101066    40
20179263    40
20183390    36
20165981    36
20143717    34
20175865    33
20180804    32
20175581    31
20120107    30
20166128    30
20126624    30
20153307    30
20176101    28
20160787    28
20188117    28
20080887    28
20159657    28
20180401    28
20174647    28
20179163    27
20050106    27
20164527    26
20198978    26
20148840    26
20181366    25
20081510    24
20110239    24
            ..
20130241     1
20060776     1
20137833     1
20060692     1
20101149     1
20135938     1
20139691     1
20147552     1
20011146     1
20157046     1
2010A417     1
20160261     1
20060093     1
20136373     1
20060357     1
20126826     1
20194342     1
20162725     1
20030094     1
20164383     1
20186651     1
20168085     1
20080014     1
20165067     1
20140126     1
20081337     1
20150321     1
20153167     1
20060077     1
19970019     1
Name: movieCd, Length: 3482, dtype: int64

In [70]:
boxoffice['영화명'].value_counts()

귀향, 끝나지 않은 이야기                 1
예수보다 낯선                        1
김복동                            1
하숙집 2                          1
망원동 인공위성                       1
극장판 공룡메카드: 타이니소어의 섬            1
스타박'스 다방                       1
비밥바룰라                          1
어떤살인                           1
상류사회                           1
프리즌                            1
마담 B                           1
미옥                             1
김광석                            1
불온한 당신                         1
워킹걸                            1
마리안느와 마가렛                      1
일진                             1
잡식가족의 딜레마                      1
우상                             1
침입자                            1
꽃보다 처녀귀신                       1
인랑                             1
엄마의 공책                         1
뷰티 인사이드                        1
왕을 참하라                         1
채비                             1
안나 카레니나                        1
맛있는 만남                         1
여자들                            1
          

In [46]:
print(writer_detail.isnull().sum())
writer_detail.sample()
writer_detail[writer_detail['sex'].isnull()]['peopleCd'].unique()

writer_detail[writer_detail['sex'].isnull()]

writer_detail[writer_detail['sex'].isnull()][['peopleCd', 'peopleNm']].drop_duplicates()

movieCd          0
movieNm          0
moviePartNm      0
peopleCd         0
peopleNm         0
sex            225
dtype: int64


,peopleCd,peopleNm
814,10057072,이인균
904,10045509,애덤 비첸
1033,10066948,조정화
1272,20214806,허선
1952,20229799,니쥔
2125,20227240,여필주
2209,20229601,권초원
2859,20200534,조성희
2994,20215586,천진우
3048,20232049,하영준


In [47]:
writer_detail[writer_detail['peopleCd'] == 10057072]

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
814,20156807,뽀로로 극장판 컴퓨터 왕국 대모험,시나리오(각본),10057072,이인균,NaN
815,20123681,뽀로로 극장판 슈퍼썰매 대모험,시나리오(각본),10057072,이인균,NaN


In [48]:
writer_code_only = pd.DataFrame(writer_detail['peopleCd'].unique(), columns=['peopleCd'])

writer_code_sex = writer_code_only.merge(writer_detail[['peopleCd', 'peopleNm', 'sex']], how='left', on='peopleCd').drop_duplicates()
writer_code_sex.sample()

,peopleCd,peopleNm,sex
9927,20238387,허준형,여자


In [49]:
writer_code_sex.groupby(['sex'])['peopleNm'].count()

sex
남자    532
여자    205
Name: peopleNm, dtype: int64

각본 공동집필이 많다
- 작품별 한명씩 보고
- 작품별 두명씩 보고

In [50]:
boxoffice.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,개봉년도
685,22,항거:유관순 이야기,2019.2.27,"9,291,145,101",1.2%,"9,291,145,101","1,157,887","1,157,887","1,094","46,156",한국,한국,(주)디씨지플러스,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,드라마,조민호,"고아성,김새벽,김예은,정하담,류경수,최무성",2019


In [51]:
staffs.sample()

,movieCd,movieNm,peopleNm,staffRoleNm
95390,20177538,완벽한 타인,이승제,VFX 슈퍼바이저


In [52]:
staffs[staffs['staffRoleNm'] == '시나리오(각본)'].groupby(['movieCd'])['peopleNm'].count()

# staffs[(staffs['staffRoleNm'] == '시나리오(각본)') & (staffs['movieCd'] == 20100211)]
staffs[(staffs['staffRoleNm'] == '시나리오(각본)') & (staffs['movieCd'] == 20120422)]

,movieCd,movieNm,peopleNm,staffRoleNm
45679,20120422,거미의 땅,김동령,시나리오(각본)
45680,20120422,거미의 땅,박경태,시나리오(각본)


In [53]:
writer_detail[writer_detail['peopleNm'] == '김동령']
writer_detail[writer_detail['peopleNm'] == '박경태']

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
4758,20190665,임신한 나무와 도깨비,감독,10028435,박경태,남자
4759,20120422,거미의 땅,감독,10028435,박경태,남자
4760,20120422,거미의 땅,시나리오(각본),10028435,박경태,남자
4761,20120422,거미의 땅,편집,10028435,박경태,남자
4762,20120422,거미의 땅,촬영,10028435,박경태,남자
4763,20090349,사당동 더하기 22,감독,10028435,박경태,남자
4764,20090349,사당동 더하기 22,촬영,10028435,박경태,남자
4765,20060444,나와 부엉이,감독,10028435,박경태,남자


In [54]:
boxoffice[boxoffice['영화명'] == '거미의 땅']

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,개봉년도
168,179,거미의 땅,2016.1.14,"6,489,100",0.0%,"7,667,100","1,025","1,291",6,113,한국,한국,(주)시네마달,(주)시네마달,15세이상관람가,다큐멘터리,"박경태,김동령",NaN,2016


각본은 순서를 알 수 없음

감독이 각본을 쓴 경우에는 boxoffice 에서 확인가능

In [55]:
# 외국인

# writer_distinct = writer_detail[writer_detail['sex'].isnull()][['peopleCd', 'peopleNm']].drop_duplicates()

# writer_distinct['Korean'] = (len(writer_distinct['peopleNm']) == 3) # 왜 안됨???

# # len(writer_distinct['peopleNm'][814])
# writer_distinct

# films.csv 에서 감독 작가 Null 값 확인

In [6]:
df = pd.read_csv('../01.Data_Collection/films.csv')

print(df.shape)
df.sample()

(839, 34)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,F-rated,감독코드,대표감독,역할_감독,감독성별
260,5.0,인천상륙작전,2016.7.27,"55,101,410,303",5.9%,"55,101,410,303","7,049,643","7,049,643","1,049","97,019",한국,한국,(주)태원엔터테인먼트,씨제이이앤엠(주),12세이상관람가,"전쟁,드라마",이재한,"김정철,이정재,이범수,리암 니슨,진세연,정준호,박철민,김병옥,김선아,김영애,박성웅,...",남기성 아들,20156250.0,20276429.0,김정철,배우,남자,"이재한, 이만희",10057235.0,이재한,시나리오(각본),남자,0,10057235.0,이재한,감독,남자


In [8]:
df.columns

Index(['순위', '영화명', '개봉일', '매출액', '매출액점유율', '누적매출액', '관객수', '누적관객수', '스크린수',
       '상영횟수', '대표국적', '국적', '제작사', '배급사', '등급', '장르', '감독', '배우', '배역명',
       '영화코드', '배우코드', '주연배우', '역할_배우', '배우성별', '작가', '작가코드', '대표작가',
       '역할_시나리오(각본)', '작가성별', 'F-rated', '감독코드', '대표감독', '역할_감독', '감독성별'],
      dtype='object')

In [9]:
# flims.csv 파일에 감독정보를 merge 하기 전으로 되돌림
film_writer = df[['순위', '영화명', '개봉일', '매출액', '매출액점유율', '누적매출액', '관객수', '누적관객수', '스크린수',
       '상영횟수', '대표국적', '국적', '제작사', '배급사', '등급', '장르', '감독', '배우', '배역명',
       '영화코드', '배우코드', '주연배우', '역할_배우', '배우성별', '작가', '작가코드', '대표작가',
       '역할_시나리오(각본)', '작가성별']].iloc[:808]

film_writer.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
641,77.0,해피 투게더,2018.11.15,"204,452,450",0.0%,"204,452,450","27,046","27,046",240,"2,322",한국,한국,(주)골든스토리픽처스,플레이리스트(주),12세이상관람가,"드라마,코미디",김정환,"박성웅,송새벽,최로운,한상혁,권해효,김건,김누리,김상균,김정수,김정현,김정환,김현정...",NaN,20189869.0,10028773.0,박성웅,배우,남자,이창열,10057674.0,이창열,시나리오(각본),남자


# 작가 Null 값 확인

In [11]:
film_writer['작가'].isnull().sum()

140

In [10]:
film_writer[film_writer['작가'].isnull()]['장르'].value_counts()

다큐멘터리                       59
드라마                         17
멜로/로맨스                      10
애니메이션                        7
멜로/로맨스,드라마                   5
공연                           4
멜로/로맨스,코미디                   3
드라마,멜로/로맨스                   3
기타,SF,드라마,코미디,판타지,멜로/로맨스     2
코미디,멜로/로맨스                   2
범죄,드라마                       2
코미디,액션                       2
멜로/로맨스,스릴러                   2
액션,어드벤처                      1
공포(호러)                       1
판타지,드라마                      1
범죄,액션                        1
뮤지컬,공연                       1
애니메이션,드라마,판타지                1
기타                           1
코미디,드라마                      1
드라마,코미디,멜로/로맨스,뮤지컬           1
애니메이션,드라마                    1
드라마,액션                       1
스릴러                          1
공포(호러),스릴러                   1
스릴러,드라마                      1
다큐멘터리,액션                     1
뮤지컬                          1
범죄,스릴러                       1
액션,드라마                       1
애니메이션,액션,어드벤처                1
미스터리,스릴러

다큐멘터리는 작가 정보 없음
- 다큐멘터리, 공연 제외(약 75개) 검색해볼 것...

In [ ]:
film_writer[film_writer['작가'].isnull()][['장르', '영화명']]

In [ ]:
df[df['영화명'] == '소년, 소녀를 만나다'] # 감독 각본 장은연

# 감독 Null 값 확인

In [311]:
# 감독 merge 테이블 가져오기
data = pd.read_csv('../01.Data_Collection/director_merged.csv')
# 감독이 각본, 제작 등 다른 파트도 맡은 경우도 함께 들어가 있어서 감독만 남겨줌
directors=data
# directors = data[data['moviePartNm'] == '감독']

print(directors.shape)
directors.sample()

directors['moviePartNm'].value_counts()
directors.groupby('movieCd')['peopleCd', 'moviePartNm'].first()['moviePartNm'].value_counts()

(2144, 9)


감독          788
그립            1
조명팀           1
배우            1
VFX 아티스트      1
스토리보드         1
Name: moviePartNm, dtype: int64

In [16]:
# movie_director_only = []
# for i in df.iloc[810:]['감독코드']:
#     print(directors.loc[directors['peopleCd'] == i, 'movieCd'])
#     movie_director_only.append(directors.loc[directors['peopleCd'] == i, 'movieCd'])

251    20151582
Name: movieCd, dtype: int64
626    20151801
Name: movieCd, dtype: int64
564    20179603
566    20152265
Name: movieCd, dtype: int64
596    20158463
Name: movieCd, dtype: int64
1099    20158526
Name: movieCd, dtype: int64
629    20161882
Name: movieCd, dtype: int64
586    20161584
587    20161904
Name: movieCd, dtype: int64
1318    20162802
Name: movieCd, dtype: int64
630    20162901
Name: movieCd, dtype: int64
448    20199346
450    20163218
Name: movieCd, dtype: int64
988    20166912
989    20164697
Name: movieCd, dtype: int64
631    20165441
Name: movieCd, dtype: int64
1049    20166652
Name: movieCd, dtype: int64
1057    20166867
1058    20162005
Name: movieCd, dtype: int64
988    20166912
989    20164697
Name: movieCd, dtype: int64
684    20167301
Name: movieCd, dtype: int64
1054    20168962
Name: movieCd, dtype: int64
786    20165001
790    20172243
Name: movieCd, dtype: int64
1402    20172963
Name: movieCd, dtype: int64
0    20177063
1    20148048
Name: movieCd, dt

In [14]:
# movie_code_in = []
# for i in movie_director_only:
#     if i in list(df['영화코드']):
#         movie_code_in.append(i)
#     else: movie_code_in.append('None')

In [17]:
directors[['movieCd', 'movieNm', 'peopleCd', 'peopleNm', 'sex']].sample()

,movieCd,movieNm,peopleCd,peopleNm,sex
727,20154905,가려진 시간,10047368,엄태화,남자


In [18]:
# film_writer['영화코드'].isnull().sum()
film_writer['영화코드'] = film_writer['영화코드'].apply(lambda x: int(x[:-2]))
film_writer['영화코드'].sample()

417    20164859
Name: 영화코드, dtype: int64

In [32]:
first_director = directors.groupby('movieCd')['peopleNm'].first().reset_index()
first_director = first_director.merge(directors[['movieCd', 'movieNm', 'peopleNm', 'peopleCd', 'sex']], how='left', on=['movieCd', 'peopleNm'])

print(first_director.shape)
first_director.sample()

(788, 5)


,movieCd,peopleNm,movieNm,peopleCd,sex
236,20156553,박은경,비밀,20066038,여자


In [76]:
data = film_writer.merge(first_director, how='outer', left_on='영화코드', right_on='movieCd')
director_merge = data.rename(columns={'peopleCd':'감독코드', 'peopleNm':'대표감독', 'sex':'감독성별'})
director_merge.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,movieCd,대표감독,movieNm,감독코드,감독성별
292,79.0,극장판 프리파라 모두 모여라! 프리즘☆투어즈,2016.10.13,"263,405,900",0.0%,"263,405,900","31,843","31,843",157,"1,796",한국,한국,동우에이앤이(주),(주)박수엔터테인먼트,전체관람가,애니메이션,히시다 마사카즈,NaN,NaN,20160261.0,NaN,NaN,NaN,NaN,아오바 죠,20259089.0,아오바 죠,시나리오(각본),남자,20160261.0,히시다 마사카즈,극장판 프리파라 모두 모여라! 프리즘☆투어즈,20129369.0,남자


In [83]:
director_merge.isnull().sum()

순위               7
영화명              0
개봉일              7
매출액              7
매출액점유율           7
누적매출액            7
관객수              7
누적관객수            7
스크린수             7
상영횟수             7
대표국적             7
국적               7
제작사             27
배급사              8
등급               7
장르               7
감독               3
배우              38
배역명            456
영화코드             0
배우코드            35
주연배우            30
역할_배우           35
배우성별            30
작가             147
작가코드           147
대표작가           147
역할_시나리오(각본)    147
작가성별           181
대표감독            27
감독코드            27
감독성별            30
dtype: int64

In [78]:
director_merge[director_merge['영화코드'].isnull()]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,movieCd,대표감독,movieNm,감독코드,감독성별
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20168962.0,유승택,입영전야 무삭제: 여선배의 자취방 버전,20234468.0,남자
809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20172243.0,김종관,최악의 하루+폴라로이드 작동법,10005660.0,남자
810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20186707.0,박범준,오늘도 위위,20306790.0,남자
811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20189862.0,김대성,감금의 시간 2 무삭제,20223386.0,남자
812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191029.0,김도영,82년생 김지영,20153842.0,여자
813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20193828.0,반태경,북간도의 십자가,20327772.0,남자
814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20197441.0,조성규,재혼의 기술,10065915.0,남자


In [79]:
director_merge[director_merge['영화코드'] == 20172243.0]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,movieCd,대표감독,movieNm,감독코드,감독성별


In [80]:
# 영화명, 영화코드 없는 데이터 입력해줌
director_merge['영화코드'] = director_merge['영화코드'].fillna(director_merge['movieCd'])
director_merge['영화명'] = director_merge['영화명'].fillna(director_merge['movieNm'])
director_merge['감독'] = director_merge['감독'].fillna(director_merge['대표감독'])

director_merge.isnull().sum()
# movieCd 컬럼 삭제
director_merge = director_merge.drop(['movieCd', 'movieNm'], axis=1)
print(director_merge.shape)
director_merge.tail()

(815, 32)


,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,대표감독,감독코드,감독성별
810,NaN,오늘도 위위,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,박범준,NaN,NaN,20186707.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,박범준,20306790.0,남자
811,NaN,감금의 시간 2 무삭제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,김대성,NaN,NaN,20189862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,김대성,20223386.0,남자
812,NaN,82년생 김지영,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,김도영,NaN,NaN,20191029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,김도영,20153842.0,여자
813,NaN,북간도의 십자가,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,반태경,NaN,NaN,20193828.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,반태경,20327772.0,남자
814,NaN,재혼의 기술,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,조성규,NaN,NaN,20197441.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,조성규,10065915.0,남자


In [81]:
director_merge['감독'].isnull().sum()

3

In [71]:
# 감독 2인 이상인 영화 있는지 확인
data = directors.groupby('movieCd')['peopleCd'].size()
movie_list = directors['movieCd'].unique()
directors_number = []

for i in director_merge[director_merge['감독'].isnull()]['영화코드'].reset_index()['영화코드']:
    if i in movie_list: 
        directors_number.append((i, data[i]))
    else: directors_number.append((i, 0))

In [72]:
directors_number

[(20189525.0, 0),
 (20184529.0, 0),
 (20185124.0, 0),
 (20168962.0, 1),
 (20172243.0, 1),
 (20186707.0, 1),
 (20189862.0, 1),
 (20191029.0, 1),
 (20193828.0, 1),
 (20197441.0, 1)]

감독 2인이상인 영화 없음

In [82]:
director_merge[director_merge['감독'].isnull()]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별,대표감독,감독코드,감독성별
499,69.0,젝스키스 에이틴,2018.1.18,"364,405,500",0.0%,"364,405,500","50,419","50,419",25,774,한국,한국,(주)와이지엔터테인먼트,씨제이 씨지브이(CJ CGV)(주),전체관람가,다큐멘터리,NaN,"은지원,이재진,장수원,김재덕,강성훈,이재진",NaN,20189525.0,10054654.0,은지원,배우,남자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,84.0,트와이스랜드,2018.12.7,"153,765,000",0.0%,"153,765,000","18,252","18,252",53,572,한국,한국,(주)제이와이피엔터테인먼트,씨제이 씨지브이(CJ CGV)(주),전체관람가,다큐멘터리,NaN,"나연,정연,모모,사나,지효,미나,다현,채영,쯔위",NaN,20184529.0,10072145.0,나연,배우,여자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,35.0,러브 유어셀프 인 서울,2019.1.26,"3,209,682,900",0.4%,"3,209,682,900","342,366","342,366",224,"4,535",한국,한국,"빅히트 엔터테인먼트,씨제이 씨지브이(CJ CGV)(주)",씨제이 씨지브이(CJ CGV)(주),전체관람가,공연,NaN,"김남준,김석진,민윤기,정호석,박지민,김태형,전정국",NaN,20185124.0,20311775.0,김남준,배우,남자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 감독 Null 값

In [123]:
data = director_merge[director_merge['감독'].notnull() & director_merge['대표감독'].isnull()]
len(data)
director_null_movie = data['영화명']
director_null_movie

89            뷰티 인사이드
90               치외법권
147            감금의 시간
188                귀향
210              입영전야
225    탐정 홍길동: 사라진 마을
228              배짱이들
234              계춘할망
264                터널
272            최악의 하루
284                대결
298      무현, 두 도시 이야기
400                하루
413            불온한 당신
416             택시운전사
500            피의 연대기
501             비밥바룰라
685        항거:유관순 이야기
802            세컨어카운트
803               자각몽
804            과대망상자들
805         소주와 아이스크림
806             히치하이커
807       소년, 소녀를 만나다
Name: 영화명, dtype: object

In [126]:
data = pd.read_csv('../01.Data_Collection/director_code.csv')

director_null_find = []
for i in director_null_movie:
    if len(data[data['movieNm'] == i]) == 0:
        director_null_find.append((i, 0))
    else: director_null_find.append((i, 1))

director_null_find

[('뷰티 인사이드', 1),
 ('치외법권', 1),
 ('감금의 시간', 0),
 ('귀향', 0),
 ('입영전야', 0),
 ('탐정 홍길동: 사라진 마을', 1),
 ('배짱이들', 1),
 ('계춘할망', 1),
 ('터널', 1),
 ('최악의 하루', 0),
 ('대결', 1),
 ('무현, 두 도시 이야기', 0),
 ('하루', 1),
 ('불온한 당신', 1),
 ('택시운전사', 1),
 ('피의 연대기', 1),
 ('비밥바룰라', 1),
 ('항거:유관순 이야기', 1),
 ('세컨어카운트', 0),
 ('자각몽', 0),
 ('과대망상자들', 0),
 ('소주와 아이스크림', 0),
 ('히치하이커', 0),
 ('소년, 소녀를 만나다', 0)]

In [160]:
data = pd.DataFrame(director_null_find, columns=['movieNm', 'in_directors'])
# data[data['in_directors'] == 0]

directors
data[data['in_directors'] == 1]['movieNm']
    

0            뷰티 인사이드
1               치외법권
5     탐정 홍길동: 사라진 마을
6               배짱이들
7               계춘할망
8                 터널
10                대결
12                하루
13            불온한 당신
14             택시운전사
15            피의 연대기
16             비밥바룰라
17        항거:유관순 이야기
Name: movieNm, dtype: object

In [161]:
code = pd.read_csv('../01.Data_Collection/director_code.csv')
director = pd.read_csv('../01.Data_Collection/directors.csv')
detail = pd.read_csv('../01.Data_Collection/director_detail.csv')

film_writer[film_writer['영화명'] == '택시운전사'] # 장훈
# directors[directors['movieNm'] == '택시운전사'] # 없음
# code[code['movieNm'] == '택시운전사'] # 있음, 장훈, 감독이 아님
# detail[detail['movieNm'] == '택시운전사'] # 있음, 장훈수-동일인물(이름이 다름) 감독이 아님
# director[director['movieNm'] == '택시운전사'] # 있음, 장훈, 감독

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
416,1.0,택시운전사,2017.8.2,"95,853,645,649",10.6%,"95,853,645,649","12,186,327","12,186,327","1,906","184,075",한국,한국,더램프(주),(주)쇼박스,15세이상관람가,드라마,장훈,"이수민,송강호,토마스 크레취만,유해진,류준열,박혁권,최귀화,이창섭,이호균,서정우,송...",광주 간호사,20162869,20289646.0,이수민,배우,여자,엄유나,20187477.0,엄유나,시나리오(각본),여자


# 작가, 감독 다시 merge 할 거임...

In [192]:
staffs = pd.read_csv('../01.Data_Collection/staffs.csv')
writer_code = pd.read_csv('../01.Data_Collection/writer_code.csv')
writer_detail = pd.read_csv('../01.Data_Collection/writer_detail.csv')
writers = staffs[staffs['staffRoleNm'] == '시나리오(각본)']

print(writers.shape)
print(writers.columns)
print(writer_code.shape)
print(writer_code.columns)
print(writer_detail.shape)
print(writer_detail.columns)

(1015, 4)
Index(['movieCd', 'movieNm', 'peopleNm', 'staffRoleNm'], dtype='object')
(1015, 5)
Index(['filmoNames', 'movieNm', 'peopleCd', 'peopleNm', 'repRoleNm'], dtype='object')
(12215, 6)
Index(['movieCd', 'movieNm', 'moviePartNm', 'peopleCd', 'peopleNm', 'sex'], dtype='object')


In [198]:
data.dtypes

movieCd         int64
movieNm        object
peopleNm       object
staffRoleNm    object
filmoNames     object
peopleCd        int64
repRoleNm      object
dtype: object

In [199]:
writer_detail.dtypes

movieCd        object
movieNm        object
moviePartNm    object
peopleCd        int64
peopleNm       object
sex            object
dtype: object

In [228]:
data = writers.merge(writer_code, how='left', on=['movieNm', 'peopleNm']).drop_duplicates()
data['movieCd'] = data['movieCd'].apply(lambda x: str(x))
print(data.shape)
# data.sample()

data = data.merge(writer_detail, how='left', on=['movieCd', 'peopleCd']).drop_duplicates()
# data[data['movieNm_x'] != data['movieNm_y']]
# data[data['peopleNm_x'] != data['peopleNm_y']]

data = data.drop('movieNm_y', axis=1)
writer_info = data.rename(columns={'movieNm_x': 'movieNm', 'peopleNm_x': 'writerNm', 'peopleNm_y':'peopleNm'})

print(writer_info.shape)
writer_info.sample()
# data['staffRoleNm'].value_counts()

# 대표 작가 | 작가 코드 | 성별
writer_first = writer_info.groupby('movieCd')['movieNm', 'writerNm', 'staffRoleNm', 'filmoNames', 'peopleCd', 'repRoleNm', 'moviePartNm', 'peopleNm', 'sex'].first().drop_duplicates().reset_index()
print(writer_first.shape)
writer_first.sample()

(961, 7)
(2069, 10)
(679, 10)


,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
543,20181142,늦여름,조성규,시나리오(각본),재혼의 기술|발광하는 현대사|늦여름|딥|각자의 미식|게스트하우스|실종2|어떻게 헤어...,10065915,감독,감독,조성규,남자


In [229]:
writer_first.isnull().sum()
writer_first[writer_first['peopleNm'].isnull()]

,movieCd,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
596,20184187,언니,김민,시나리오(각본),내 처제가 된 첫사랑|정사 2018|노래방 : 화끈한 여자들|유부녀들의 맛|밀애 :...,20269709,동시녹음,NaN,NaN,NaN


In [223]:
film_writer[film_writer['영화코드'] == 20184187]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,작가코드,대표작가,역할_시나리오(각본),작가성별
666,46.0,언니,2019.1.1,"1,746,373,250",0.2%,"1,757,528,250","197,134","198,377",530,"11,315",한국,한국,"필름에이픽쳐스,(유)조이앤시네마","(주)제이앤씨미디어그룹,TCO(주)더콘텐츠온",청소년관람불가,액션,임경택,"이시영,박세완,이준혁,최진호",NaN,20184187,10056359.0,이시영,배우,여자,NaN,NaN,NaN,NaN,NaN


언니 작가 김민 영화인코드: 20312772, 성별은 Null

In [255]:
writer_info['movieCd'].value_counts()
writer_info[writer_info['movieCd'] == '20161568']
len(writer_info[writer_info['moviePartNm'] == '시나리오(각본)']['movieCd'])
writer_info['moviePartNm'].value_counts()
# 같은 사람이 여러 역할로 중복되어 들어가있음
# moviePartNm 이 시나리오(각본) 인것만 남긴다
# moviePartNm 이 시나리오(각본)이 아닌데 하나만 있는 경우 있는지 확인

writer_info = writer_info[writer_info['moviePartNm'] == '시나리오(각본)']

In [244]:
writer_first['moviePartNm'].value_counts()
len(writer_first[['movieCd', 'peopleCd']].drop_duplicates()) 
# 중복으로 들어간 사람 없다
# moviePartNm 이 시나리오가 아닌 경우 있다

679

In [256]:
# 작가 (전체 리스트)

movieCd_list = writer_info['movieCd'].unique()
# 영화코드를 바꿔준다

writer_list = []
for movieCd in movieCd_list:
    writers_of_movie = ''

    data = writer_info[writer_info['movieCd'] == movieCd]
    for writerNm in data['writerNm']:
        if writers_of_movie != '': writers_of_movie += ', '
        writers_of_movie += writerNm
        
    writer_list.append((movieCd, writers_of_movie))

writer_list

[('20148048', '류승완'),
 ('20148851', '이기철, 최동훈'),
 ('20149314', '우민호'),
 ('20145503', '장재현'),
 ('20148846', '조철현, 오승현, 이송원'),
 ('20148845', '이병헌'),
 ('20136068', '수오, 민지은'),
 ('20149570', '김수진, 이남규'),
 ('20148855', '한대덕, 곽경택'),
 ('20149569', '백운학'),
 ('20156557', '황혜순, 김정훈'),
 ('20157464', '김봉주'),
 ('20136888', '유하'),
 ('20140703', '이상현'),
 ('20143572', '김선정, 노경희, 박정예'),
 ('20148188', '김현석'),
 ('20142407', '박진표, 이병헌'),
 ('20149266', '한준희'),
 ('20142402', '이윤성, 민규동'),
 ('20145171', '박훈정'),
 ('20157465', '윤준형, 김영갑'),
 ('20142855', '이공주, 최관영'),
 ('20148461', '손용호, 안영진'),
 ('20145504', '노덕'),
 ('20156748', '서유민, 정기훈'),
 ('20143513', '김제영'),
 ('20131262', '하정우, 김주호'),
 ('20148841', '조사무엘, 신한솔'),
 ('20130566', '오승욱'),
 ('20136063', '김세희'),
 ('20148844', '김광태'),
 ('20148553', '최정미, 문제용, 유선동'),
 ('20140144', '김아영, 이종필'),
 ('20140193', '김익로, 천성일'),
 ('20143511', '최윤진'),
 ('20141224', '이해영'),
 ('20143512', '조중훈'),
 ('20157501', '하기호'),
 ('20136057', '손아람'),
 ('20130021', '박흥식, 최아름'),
 ('20155826'

In [258]:
writer_list = pd.DataFrame(writer_list, columns=['movieCd', '작가'])

print(writer_list.shape)
writer_list.sample()

(670, 2)


,movieCd,작가
190,20167867,김기덕


In [260]:
writer_part = writer_list.merge(writer_first, how='outer', on='movieCd')
print(writer_part.shape)
writer_part.sample()

(679, 11)


,movieCd,작가,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
421,20179205,"조은이, 이남규, 한증애",조선명탐정: 흡혈괴마의 비밀,조은이,시나리오(각본),조선명탐정: 흡혈괴마의 비밀|우는 남자|캠퍼스 S 커플|추격자,20135974,시나리오(각본),시나리오(각본),조은이,여자


In [263]:
writer_part['moviePartNm'].value_counts()

감독          405
시나리오(각본)    237
배우           11
프로듀서          6
기획            4
제작            4
원작            2
그립            1
조감독           1
광고디자인         1
VFX 아티스트      1
스크립터          1
편집            1
특수촬영          1
연출팀           1
스토리보드         1
Name: moviePartNm, dtype: int64

# [boxoffice - 배우] 테이블에 작가 merge

In [293]:
# flims.csv 파일에 작가정보를 merge 하기 전으로 되돌림
film_actor = df[['순위', '영화명', '개봉일', '매출액', '매출액점유율', '누적매출액', '관객수', '누적관객수', '스크린수',
       '상영횟수', '대표국적', '국적', '제작사', '배급사', '등급', '장르', '감독', '배우', '배역명',
       '영화코드', '배우코드', '주연배우', '역할_배우', '배우성별']].iloc[:808]

# merge 할 테이블의 영화코드는 str 이라 맞춰준다
film_actor['영화코드'] = film_actor['영화코드'].apply(lambda x: str(x)[:-2])
film_actor.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별
607,93.0,나비잠,2018.9.6,"89,716,950",0.0%,"89,716,950","10,605","10,605",91,"1,618",한국,"한국,일본",(주)영화사조아,(주)트리플픽쳐스,15세이상관람가,멜로/로맨스,정재은,"나카야마 미호,김재욱,마시마 히데카즈",NaN,20175587,10007199.0,나카야마 미호,배우,여자


In [323]:
film_writer = film_actor.merge(writer_part, how='outer', left_on='영화코드', right_on='movieCd').drop_duplicates()
print(film_writer.shape)
film_writer.sample()
film_writer.isnull().sum()

(810, 35)


순위               2
영화명              2
개봉일              2
매출액              2
매출액점유율           2
누적매출액            2
관객수              2
누적관객수            2
스크린수             2
상영횟수             2
대표국적             2
국적               2
제작사             22
배급사              3
등급               2
장르               2
감독               5
배우              33
배역명            451
영화코드             2
배우코드            30
주연배우            25
역할_배우           30
배우성별            25
movieCd        131
작가             140
movieNm        131
writerNm       131
staffRoleNm    131
filmoNames     131
peopleCd       131
repRoleNm      131
moviePartNm    132
peopleNm       132
sex            166
dtype: int64

# 박스오피스 데이터에 없는 영화명, 영화코드 넣어주기

In [324]:
film_writer[film_writer['영화명'].isnull()]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,movieCd,작가,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20183144,윤용진,선종 무문관,윤용진,시나리오(각본),선종 무문관|할|내 마음 속에 작은 영웅들,20110475.0,감독,감독,윤용진,남자
809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20160501,김의곤,두 번째 겨울,김의곤,시나리오(각본),두 번째 겨울,20244869.0,감독,감독,김의곤,남자


In [325]:
film_writer['영화명'] = film_writer['영화명'].fillna(film_writer['movieNm'])
film_writer['영화코드'] = film_writer['영화코드'].fillna(film_writer['movieCd'])

film_writer[film_writer['순위'].isnull()]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,movieCd,작가,movieNm,writerNm,staffRoleNm,filmoNames,peopleCd,repRoleNm,moviePartNm,peopleNm,sex
808,NaN,선종 무문관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20183144,NaN,NaN,NaN,NaN,20183144,윤용진,선종 무문관,윤용진,시나리오(각본),선종 무문관|할|내 마음 속에 작은 영웅들,20110475.0,감독,감독,윤용진,남자
809,NaN,두 번째 겨울,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20160501,NaN,NaN,NaN,NaN,20160501,김의곤,두 번째 겨울,김의곤,시나리오(각본),두 번째 겨울,20244869.0,감독,감독,김의곤,남자


# 컬럼명 수정

In [326]:
data = film_writer.drop(['movieCd', 'movieNm', 'moviePartNm', 'staffRoleNm'], axis=1)
film_writer = data.rename(columns={'writerNm':'대표작가(필명)', 'peopleNm':'대표작가', 'peopleCd':'작가코드','repRoleNm':'작가_대표역할', 'sex':'작가성별', 'filmoNames':'작가필모'})

film_writer.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,대표작가(필명),작가필모,작가코드,작가_대표역할,대표작가,작가성별
416,1.0,택시운전사,2017.8.2,"95,853,645,649",10.6%,"95,853,645,649","12,186,327","12,186,327","1,906","184,075",한국,한국,더램프(주),(주)쇼박스,15세이상관람가,드라마,장훈,"이수민,송강호,토마스 크레취만,유해진,류준열,박혁권,최귀화,이창섭,이호균,서정우,송...",광주 간호사,20162869,20289646.0,이수민,배우,여자,엄유나,엄유나,말모이|택시운전사|번개와 춤을,20187477.0,시나리오(각본),엄유나,여자


# [boxoffice - 배우 - 작가] 테이블에 감독 정보 merge

In [299]:
print(directors.shape)
directors.sample()

(2144, 9)


,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex,repRoleNm_x,filmoNames,repRoleNm_y
511,20152368,인 허 플레이스,감독,20212728,알버트 신,남자,감독,인 허 플레이스,감독


In [345]:
directors[directors['repRoleNm_x'] != directors['repRoleNm_y']] # 하정우만 다름(배우/감독)

# 대표 감독 | 감독 코드 | 성별
first_director = directors.groupby(['movieCd', 'movieNm'])['peopleCd', 'peopleNm', 'repRoleNm_y','filmoNames', 'sex'].first().reset_index()

# first_director = first_director.drop('repRoleNm_x', axis=1)
first_director = first_director.rename(columns={'repRoleNm_y':'감독_대표역할', 'movieCd':'영화코드', 'movieNm':'영화명', 'peopleCd':'감독코드', 'peopleNm':'대표감독', 'filmoNames':'감독필모', 'sex':'감독성별'})
first_director['영화코드'] = first_director['영화코드'].apply(lambda x: str(x))
first_director.sample()

,영화코드,영화명,감독코드,대표감독,감독_대표역할,감독필모,감독성별
222,20155388,초인,20168380,서은영,감독,고백|초인|장마,여자


In [346]:
film_director = film_writer.merge(first_director, how='outer', on='영화코드').drop_duplicates()
print(film_director.shape)
film_director.sample()

(817, 37)


,순위,영화명_x,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,대표작가(필명),작가필모,작가코드,작가_대표역할,대표작가,작가성별,영화명_y,감독코드,대표감독,감독_대표역할,감독필모,감독성별
89,15.0,뷰티 인사이드,2015.8.20,"16,148,479,531",1.8%,"16,148,479,531","2,053,435","2,053,435",648,"52,618",한국,한국,(주)용필름,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,"판타지,멜로/로맨스",백,"한효주,김대명,도지한,배성우,박신혜,이범수,박서준,김상호,천우희,우에노 주리,이재준...",홍이수,20143572,10087820.0,한효주,배우,여자,"김선정, 노경희, 박정예",김선정,뷰티 인사이드|채식주의자|국가대표|미녀는 괴로워|사랑 따윈 필요 없어,10004533.0,각색,김선정,여자,NaN,NaN,NaN,NaN,NaN,NaN


In [347]:
film_director.isnull().sum()
film_director[film_director['영화명_x'] != film_director['영화명_y']]

film_director['영화명_x'] = film_director['영화명_x'].fillna(film_director['영화명_y'])
film_director['영화명_x'].isnull().sum()

film_director = film_director.drop('영화명_y', axis=1)
film_director = film_director.rename(columns={'영화명_x':'영화명'})
film_director.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,대표작가(필명),작가필모,작가코드,작가_대표역할,대표작가,작가성별,감독코드,대표감독,감독_대표역할,감독필모,감독성별
639,180.0,인어전설,2018.11.15,"8,698,250",0.0%,"8,698,250","1,113","1,113",14,183,한국,한국,주식회사 자파리필름,(주)미로스페이스,12세이상관람가,"코미디,드라마",오멸,"전혜빈,문희경,이경준,강래연,윤효진,이경준,이경진,이경호,이동규,이동현,이수연,이순...",NaN,20174008,10061532.0,전혜빈,배우,여자,"오멸, 전병원",오멸,인어전설|눈꺼풀|하늘의 황금마차|지슬 - 끝나지 않은 세월2|뽕똘|이어도|어이그 저...,10050307.0,감독,오멸,남자,10050307.0,오멸,감독,인어전설|눈꺼풀|하늘의 황금마차|지슬 - 끝나지 않은 세월2|뽕똘|이어도|어이그 저...,남자


In [348]:
film_director['역할_배우'].value_counts()

배우    778
감독      2
Name: 역할_배우, dtype: int64

In [336]:
pd.read_csv('../01.Data_Collection/actor_leading_role.csv')['repRoleNm'].value_counts()

배우    770
Name: repRoleNm, dtype: int64

In [349]:
print(film_director.shape)
film_director.isnull().sum()

(817, 36)


순위            9
영화명           0
개봉일           9
매출액           9
매출액점유율        9
누적매출액         9
관객수           9
누적관객수         9
스크린수          9
상영횟수          9
대표국적          9
국적            9
제작사          29
배급사          10
등급            9
장르            9
감독           12
배우           40
배역명         458
영화코드          0
배우코드         37
주연배우         32
역할_배우        37
배우성별         32
작가          147
대표작가(필명)    138
작가필모        138
작가코드        138
작가_대표역할     138
대표작가        139
작가성별        173
감독코드         24
대표감독         24
감독_대표역할      24
감독필모         24
감독성별         27
dtype: int64

In [342]:
print(df.shape)
df.isnull().sum()

(839, 34)


순위              31
영화명              0
개봉일             31
매출액             31
매출액점유율          31
누적매출액           31
관객수             31
누적관객수           31
스크린수            31
상영횟수            31
대표국적            31
국적              31
제작사             51
배급사             32
등급              31
장르              31
감독              34
배우              62
배역명            480
영화코드             0
배우코드            59
주연배우            54
역할_배우           59
배우성별            54
작가             169
작가코드           169
대표작가           169
역할_시나리오(각본)    169
작가성별           203
F-rated          0
감독코드            51
대표감독            51
역할_감독           51
감독성별            53
dtype: int64

In [350]:
film_director.to_csv('../01.Data_Collection/films.csv', index=False)

In [351]:
pd.read_csv('../01.Data_Collection/films.csv')

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,대표작가(필명),작가필모,작가코드,작가_대표역할,대표작가,작가성별,감독코드,대표감독,감독_대표역할,감독필모,감독성별
0,55.0,워킹걸,2015.1.7,"1,183,887,479",0.1%,"1,201,963,479","149,181","151,550",308,"7,076",한국,한국,"(주)홍필름,(주)수필름",메가박스중앙(주)플러스엠,청소년관람불가,"코미디,멜로/로맨스",정범식,"조여정,클라라,김태우,배유람,정범식,정범식,박주희,백경인,조은영,손진우,배성우,유찬...",NaN,20147947,10066680.0,조여정,배우,여자,정범식,정범식,공포체험단 곤지암편|천문: 하늘에 묻는다|곤지암|워킹걸|미쓰GO|외톨이|기담|무서운...,10061758.0,감독,정범식,남자,10061758.0,정범식,감독,공포체험단 곤지암편|천문: 하늘에 묻는다|곤지암|워킹걸|미쓰GO|외톨이|기담|무서운...,남자
1,121.0,설해,2015.1.8,"20,767,500",0.0%,"29,770,500","2,796","3,933",34,335,한국,한국,(주)맑음영화사,"(주)마인스 엔터테인먼트,주식회사 마운틴픽쳐스",12세이상관람가,"멜로/로맨스,드라마",김정권,"박해진,이영아,송재희,전무송,최원주,이상민,이기열,윤기호,김수환,김정권,김혜진,박재...",상우,20124398,20126329.0,박해진,배우,남자,"김정권, 김군영",김정권,설해|그 남자의 책 198쪽|바보|화성으로 간 사나이|동감|귀소,10005546.0,감독,김정권,남자,10005546.0,김정권,감독,설해|그 남자의 책 198쪽|바보|화성으로 간 사나이|동감|귀소,남자
2,26.0,허삼관,2015.1.14,"7,408,943,569",0.8%,"7,408,943,569","955,679","955,679",621,"27,645",한국,한국,"(주)두타연,(주)판타지오픽쳐스",(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,드라마,하정우,"하정우,하지원,전혜진,장광,주진모,성동일,이경영,김영애,정만식,조진웅,김기천,김성균...",허삼관,20131262,10087253.0,하정우,배우,남자,"하정우, 김주호",하정우,백두산|PMC: 더 벙커|클로젯|신과함께-인과 연|1987|신과함께-죄와 벌|서서평...,10087253.0,배우,하정우,남자,10087253.0,하정우,배우,백두산|PMC: 더 벙커|클로젯|신과함께-인과 연|1987|신과함께-죄와 벌|서서평...,남자
3,16.0,오늘의 연애,2015.1.14,"14,924,654,487",1.7%,"14,924,654,487","1,891,993","1,891,993",709,"47,237",한국,한국,(주)팝콘에프앤엠,씨제이이앤엠(주),15세이상관람가,"멜로/로맨스,코미디",박진표,"이승기,문채원,이서진,장진우,이경진,이동진,임하룡,이상협,김광규,김부선,가인,임종윤...",NaN,20142407,10056227.0,이승기,배우,남자,"박진표, 이병헌",박진표,오늘의 연애|공범|내 사랑 내 곁에|그놈 목소리|너는 내 운명|죽어도 좋아|신비한 ...,10029363.0,감독,박진표,남자,10029363.0,박진표,감독,오늘의 연애|공범|내 사랑 내 곁에|그놈 목소리|너는 내 운명|죽어도 좋아|신비한 ...,남자
4,149.0,꽃보다 처녀귀신,2015.1.15,"7,909,000",0.0%,"7,909,000","1,567","1,567",5,42,한국,한국,고구마콘텐츠허브(주),고구마콘텐츠허브(주),청소년관람불가,기타,최영민,"안재민,이정민,이정민,박승찬,이현복,최경택,손지훈,우경희,유재훈,이돌,하혜정,한상민...",NaN,20155744,20204727.0,안재민,배우,남자,윤영호,윤영호,꽃보다 처녀귀신|태양의 이면|바이칼|바르도|삼승할망|좀비처럼 걸어봐,10054419.0,감독,윤영호,남자,10072470.0,최영민,촬영,공사중|꽃보다 처녀귀신|일탈여행: 프라이빗 아일랜드|영혼의 부활|스타: 빛나는 사랑...,남자
5,113.0,미라클 여행기,2015.1.15,"20,562,500",0.0%,"21,417,500","3,228","3,399",12,320,한국,한국,"미라클필름,(주)꿈길제작소,허철필름",미라클필름,전체관람가,다큐멘터리,허철,"최미라,김영민,김정민",여행자,20148631,20188866.0,최미라,배우,여자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20114576.0,허철,감독,돌아온다|필름시대사랑|미라클 여행기|영화판,남자
6,119.0,고양이 장례식,2015.1.15,"23,200,900",0.0%,"23,200,900","2,865","2,865",46,394,한국,한국,영화 고양이 장례식 제작 조합,(주)인디플러그,12세이상관람가,멜로/로맨스,이종훈,"강인,박세영,정겨운,강신유,신무호,김민지,이민희,남기준,민지,김병춘,김도영,차민지",NaN,20144601,10005001.0,강인,배우,남자,이종훈,이종훈,고양이 장례식|심판의날,10057421.0,감독,이종훈,남자,10057421.0,이종훈,감독,고양이 장례식|심판의날,남자
7,14.0,강남 1970,2015.1.21,"17,718,699,499",2.0%,"17,718,699,499","2,192,276","2,192,276",841,"53,304",한국,한국,"(주)모베라픽처스,(주)쇼박스",(주)쇼박스,청소년관람불가,"액션,드라마",유하,"이민호,김래원,정진영,설현,유승목,한재영,이연두,정호빈,최진호,엄효섭,최병모,지대한...",종대,20136888,20185376.0,이민호,배우,남자,유하,유하,파이프라인|강남 1970|강남1970(무삭제판)|하울링|쌍화점|비열한 거리|말죽거리...,20111250.0,감독,유하,남자,20111250.0,유하,감독,파이프라인|강남 1970|강남1970(무삭제판)|하울링|쌍화점|비열한 거리|말죽거리...,남자
8,111.0,생각보다 맑은,2015.1.22,"27,393,900",0.0%,"27,393,900","3,881","3,881",17,458,한국,한국,Studio Allo,"애니메이션 배급, 씨앗,㈜인디스토리",12세이상관람가,애니메이션,한지원,"엄상현,양정화,한지원,한지원,이홍수,이홍수,이홍수,이호민,이영기,유아람,서정원,서정원",NaN,20145202,10047326.0,엄상현,배우,남자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20180788.0,한지원,감독,생각보다 맑은|26년|학교 가는 길|코피루왁|[작은영화영화제]인디애니페스트 2018...,여자
9,41.0,내 심장을 쏴라,2015.1.28,"2,856,859,132",0.3%,"2,856,859,132","387,409","387,409",452,"20,042",한국,한국,(주)주피터필름,"(주)리틀빅픽쳐스,(주)이수C&E",15세이상관람가,드라마,문제용,"이민기,여진구,유오성,김정태,김기천,신구,송영창,박충선,박두식,한혜린,이준혁,임문영...",NaN,20148553,10055447.0,이민기,배우,남자,"최정미, 문제용, 유선동",최정미,"고산자, 대동여지도|따라지 : 비열한 거리|내 심장을 쏴라",20209326.0,시나리오(각본),최정미,여자,10026804.0,문제용,감독,내 심장을 쏴라|아부의 왕|슈퍼맨이었던 사나이|타짜|내 생애 가장 아름다운 일주일|...,남자


In [355]:
film_director[film_director['대표감독'].isnull()]

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,배역명,영화코드,배우코드,주연배우,역할_배우,배우성별,작가,대표작가(필명),작가필모,작가코드,작가_대표역할,대표작가,작가성별,감독코드,대표감독,감독_대표역할,감독필모,감독성별
89,15.0,뷰티 인사이드,2015.8.20,"16,148,479,531",1.8%,"16,148,479,531","2,053,435","2,053,435",648,"52,618",한국,한국,(주)용필름,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,"판타지,멜로/로맨스",백,"한효주,김대명,도지한,배성우,박신혜,이범수,박서준,김상호,천우희,우에노 주리,이재준...",홍이수,20143572,10087820.0,한효주,배우,여자,"김선정, 노경희, 박정예",김선정,뷰티 인사이드|채식주의자|국가대표|미녀는 괴로워|사랑 따윈 필요 없어,10004533.0,각색,김선정,여자,NaN,NaN,NaN,NaN,NaN
90,44.0,치외법권,2015.8.27,"2,664,941,005",0.3%,"2,664,941,005","346,483","346,483",446,"15,297",한국,한국,"(주)유니프로,(주)스톰픽쳐스코리아",판씨네마(주),15세이상관람가,액션,신동엽,"임창정,최다니엘,임은경,장광,이경영,문제영,정한비,박영진,임정아,박일우,정형석,조원...",NaN,20155826,10058849.0,임창정,배우,남자,민경근,민경근,대결|치외법권,20204447.0,시나리오(각본),민경근,남자,NaN,NaN,NaN,NaN,NaN
147,242.0,감금의 시간,2015.12.3,"3,310,000",0.0%,"3,310,000",555,555,20,49,한국,한국,마법사필름,마법사필름,청소년관람불가,"스릴러,드라마",김대성,"전유진,허동원,김윤재,이용주",현아,20158904,20223387.0,전유진,배우,여자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,12.0,귀향,2016.2.24,"27,249,738,990",2.9%,"27,249,738,990","3,587,173","3,587,173",876,"90,286",한국,한국,주식회사 제이오엔터테인먼트코리아,(주)와우픽쳐스,15세이상관람가,드라마,조정래,"손숙,강하나,서미지,최리,정인기,오지혜,김시은,이채현,임성철,임혜영,박주현,박지희,...",영옥(영희),20161364,10036856.0,손숙,배우,여자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,263.0,입영전야,2016.4.5,"3,290,000",0.0%,"3,290,000",492,492,20,46,한국,한국,영화사 인연,(주)소나무픽쳐스,청소년관람불가,"멜로/로맨스,코미디",유승택,"임형순,이도현,이선영,이소영,이윤희",NaN,20162701,20248536.0,임형순,배우,여자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,423.0,배짱이들,2016.5.12,"1,521,600",0.0%,"1,521,600",210,210,6,111,한국,한국,뉴에이지필름,주식회사 씨엠닉스,12세이상관람가,다큐멘터리,정훈,"이인세,강백수,최휘찬,송형준,정솔,문용일,이경호,이국현,김균현,김두복,김병진,김진규",NaN,20153803,20214367.0,이인세,배우,남자,박은하,박은하,배짱이들,20215087.0,시나리오(각본),박은하,여자,NaN,NaN,NaN,NaN,NaN
234,36.0,계춘할망,2016.5.19,"3,600,794,348",0.4%,"3,600,794,348","482,516","482,516",526,"19,644",한국,한국,"지오엔터테인먼트,(주)퍼플켓츠필름,(주)빅스토리픽쳐스",(주)콘텐츠 난다긴다,15세이상관람가,"가족,드라마",창,"윤여정,김고은,김희원,신은정,최민호,류준열,양익준,박민지,남태부,강경탁,안지혜,이상...",NaN,20156559,10054391.0,윤여정,배우,여자,"양서현, 허아름",양서현,계춘할망,20209267.0,시나리오(각본),양서현,여자,NaN,NaN,NaN,NaN,NaN
264,4.0,터널,2016.8.10,"57,529,484,417",6.2%,"57,529,484,417","7,120,508","7,120,508","1,105","113,201",한국,한국,"(주)어나더썬데이,(주)비에이엔터테인먼트",(주)쇼박스,12세이상관람가,드라마,김성훈,"하정우,배두나,오달수,김종수,임한빈,정민희,송민석,윤성민,이동진,이상희,최춘범,권승...",NaN,20154910,10087253.0,하정우,배우,남자,NaN,김성훈,백두산|PMC: 더 벙커|클로젯|신과함께-인과 연|1987|신과함께-죄와 벌|서서평...,10087253.0,배우,하정우,남자,NaN,NaN,NaN,NaN,NaN
272,61.0,최악의 하루,2016.8.25,"639,959,190",0.1%,"639,959,190","80,754","80,754",199,"5,606",한국,한국,㈜인디스토리,CGV아트하우스,15세이상관람가,멜로/로맨스,김종관,"한예리,이와세 료,권율,이희준,이승연,강 숙,최유화,이승연",은희,20155321,20090447.0,한예리,배우,여자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,78.0,대결,2016.9.22,"251,176,500",0.0%,"251,176,500","32,278","32,278",366,"5,409",한국,한국,"휴메니테라픽쳐스(주),컴퍼니에이이엔티(주)","(주)스톰픽쳐스코리아,TCO(주)더콘텐츠온",15세이상관람가,액션,신동엽,"이주승,오지호,이정진,신정근,손은서,임효우,박주혜,박준범,전혜인,이규희,이대겸,조훈...",NaN,20162022,10057434.0,이주승,배우,남자,"민경근, 정성훈, 이현철",민경근,대결|치외법권,20204447.0,시나리오(각본),민경근,남자,NaN,NaN,NaN,NaN,NaN


In [353]:
film_director['장르'].value_counts()

드라마                   194
다큐멘터리                 134
멜로/로맨스                 40
애니메이션                  39
코미디                    28
스릴러                    24
멜로/로맨스,드라마             23
코미디,드라마                17
드라마,코미디                15
범죄,드라마                 14
범죄,액션                  14
액션                     14
미스터리,스릴러               13
멜로/로맨스,코미디             12
사극                     10
액션,드라마                  9
드라마,멜로/로맨스              8
판타지,드라마                 7
스릴러,드라마                 7
공포(호러),스릴러              7
공포(호러)                  6
드라마,가족                  6
범죄,스릴러                  6
미스터리,드라마                5
가족,드라마                  5
멜로/로맨스,스릴러              5
범죄                      5
사극,드라마                  5
코미디,멜로/로맨스              4
코미디,액션                  4
                     ... 
드라마,다큐멘터리               1
판타지,가족                  1
코미디,가족                  1
멜로/로맨스,기타               1
사극,코미디,어드벤처             1
SF,액션                   1
액션,사극                   1
코미디,사극      

In [357]:
df = film_director

In [364]:
df['상영횟수'].dtypes
df['상영횟수'] = df['상영횟수'].apply(lambda x: int(x.replace(',', '')) if x.notnull() else: x)
# df['상영횟수'].hist()

SyntaxError: invalid syntax (<ipython-input-364-6e3c99ac097b>, line 2)

# missing 작가, 감독 영화인 상세정보 추가 

In [6]:
director_code = pd.read_csv('../01.Data_Collection/directorCd_missing.csv')
director_detail = pd.read_csv('../01.Data_Collection/director_detail_missing.csv')
print(director_code.shape)
print(director_detail.shape)
writer_code = pd.read_csv('../01.Data_Collection/writerCd_missing.csv')
writer_detail = pd.read_csv('../01.Data_Collection/writer_detail_missing.csv')
print(writer_code.shape)
print(writer_detail.shape)

(21, 5)
(358, 6)
(7, 5)
(47, 6)


## 감독 merge

In [7]:
director_code.sample()

,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
15,로마서 8:37|동주|프랑스 영화처럼|조류인간|개를 훔치는 완벽한 방법|배우는 배우...,과대망상자들,10040499,신연식,감독


In [8]:
director_detail.sample()

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
77,20182404,착한 친구엄마,조감독,20234468,유승택,남자


In [75]:
data = director_code.merge(director_detail, how='left', on=['movieNm', 'peopleCd'])
# .groupby(['movieCd', 'peopleCd']).first()
print(data.shape)
data.sample()

data = data[data['peopleNm_y'].notnull()].drop_duplicates()
# len(data['movieCd'].unique())
# len(data[data['moviePartNm'] == '감독']['movieCd'])
director_merge = data[data['moviePartNm'] == '감독']
director_merge = director_merge.drop('peopleNm_x', axis=1)
director_merge = director_merge.rename(columns={'peopleNm_y':'peopleNm'})
print(director_merge.shape)
director_merge.sample()

(48, 9)
(16, 8)


,filmoNames,movieNm,peopleCd,repRoleNm,movieCd,moviePartNm,peopleNm,sex
27,사이코메트리|평행이론|네번째 층 - 어느날 갑자기 두번째 이야기|풍경|자각몽|그의 ...,자각몽,10002500,감독,20161411.0,감독,권호영,남자


In [ ]:
data[data['peopleNm_y'].notnull()].drop_duplicates()['movieCd']

In [45]:
director_detail[director_detail['movieNm'] == '배짱이들']
director_detail[director_detail['movieNm'] == '계춘할망']
director_detail[director_detail['movieNm'] == '터널']
director_detail[director_detail['movieNm'] == '불온한 당신']
director_detail[director_detail['movieNm'] == '택시운전사']

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
214,20162869,택시운전사,감독,10060674,장훈,남자


merge 하면서 누락된 위 다섯개 작품 감독들을 살펴보니 영화인코드가 다르다...

In [70]:
# 영화인 코드가 다른 감독들을 직접 찾아줌
director_movieNm = ['배짱이들'
                    , '계춘할망'
                    , '터널'
                    , '불온한 당신'
                    , '택시운전사'
]

director_peopleNm = ['정훈'
                     , '창'
                     , '김성훈'
                     , '이영'
                     , '장훈']

director_filmo = ['배짱이들'
                  , '숲속의 아이|계춘할망|치명도수: RESET| 표적|48미터|고死: 피의 중간고사'
                  , '터널|끝까지 간다|애정결핍이 두 남자에게 미치는 영향|그놈은 멋있었다|오! 해피데이'
                  , '불온한 당신|폐허의 골격|Out: 이반검열 두 번째 이야기|우리들은 정의파다|이반검열 1|거북이 시스터즈'
                  , '택시운전사|고지전|의형제|영화는 영화다|시간|활|빈집|신부수업|사마리아|헤라퍼플']

director_neg = pd.DataFrame(director_movieNm, director_peopleNm).reset_index().rename(columns={'index':'peopleNm', 0:'movieNm'})
director_neg['filmoNames'] = director_filmo
data = director_neg.merge(director_detail, how='left', on=['peopleNm', 'movieNm'])
director_neg = data[data['moviePartNm'] == '감독']
director_neg

,peopleNm,movieNm,filmoNames,movieCd,moviePartNm,peopleCd,sex
0,정훈,배짱이들,배짱이들,20153803,감독,20214366,남자
1,창,계춘할망,숲속의 아이|계춘할망|치명도수: RESET| 표적|48미터|고死: 피의 중간고사,20156559,감독,10071587,남자
4,김성훈,터널,터널|끝까지 간다|애정결핍이 두 남자에게 미치는 영향|그놈은 멋있었다|오! 해피데이,20154910,감독,10004662,남자
6,이영,불온한 당신,불온한 당신|폐허의 골격|Out: 이반검열 두 번째 이야기|우리들은 정의파다|이반검...,20154427,감독,10056635,여자
12,장훈,택시운전사,택시운전사|고지전|의형제|영화는 영화다|시간|활|빈집|신부수업|사마리아|헤라퍼플,20162869,감독,10060674,남자


In [78]:
director_add = pd.concat([director_merge, director_neg], ignore_index=True)
director_add['repRoleNm'] = director_add['repRoleNm'].fillna('감독')
director_add

,filmoNames,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,repRoleNm,sex
0,뷰티 인사이드|설국열차|커플즈|그놈 목소리|연애의 목적|S 다이어리|올드보이,20143572.0,뷰티 인사이드,감독,20143077,백종열,감독,남자
1,응징자2|게이트-완결판|게이트|대결|치외법권|따라지: 비열한 거리 무삭제판|따라지 ...,20155826.0,치외법권,감독,20111176,신재호,감독,남자
2,여선생2 무삭제|우리도련님 무삭제|우리 도련님|여선생2|여자친구|감금의 시간 2 무...,20158904.0,감금의 시간,감독,20223386,김대성,감독,남자
3,"에움길|소리꾼|귀향, 끝나지 않은 이야기|귀향|파울볼|두레소리",20161364.0,귀향,감독,10066934,조정래,감독,남자
4,착한 이모의 은밀한 아르바이트|성인용품 사용설명서|착한 이모의 은밀한 아르바이트 무...,20162701.0,입영전야,감독,20234468,유승택,감독,남자
5,아무도 없는 곳|더 테이블|최악의 하루|조금만 더 가까이|바람의 노래|엄마 찾아 삼...,20155321.0,최악의 하루,감독,10005660,김종관,감독,남자
6,응징자2|게이트-완결판|게이트|대결|치외법권|따라지: 비열한 거리 무삭제판|따라지 ...,20162022.0,대결,감독,20111176,신재호,감독,남자
7,"시민 노무현|무현, 두 도시 이야기 : 파이널 컷|무현, 두 도시 이야기|품행제로|...",20162545.0,"무현, 두 도시 이야기",감독,20153728,전인환,감독,남자
8,관능의 법칙|원더풀 라디오|참을 수 없는|뜨거운 것이 좋아|싱글즈|이공|봉자|사랑하...,20161410.0,세컨어카운트,감독,10002537,권칠인,감독,남자
9,사이코메트리|평행이론|네번째 층 - 어느날 갑자기 두번째 이야기|풍경|자각몽|그의 ...,20161411.0,자각몽,감독,10002500,권호영,감독,남자


In [ ]:
## films.csv 다시 받아서 붙이기

## 작가 merge

In [84]:
print(writer_code.shape)
writer_code
# .sample()

# print(writer_detail.shape)
# writer_detail.sample()

(7, 5)


,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
0,태양을 쏴라|이중간첩,태양을 쏴라,10005627,김정행,기획진행
1,유부녀들,유부녀들,20227564,신정규,시나리오(각본)
2,위선자들|캐치미|슈퍼맨 강보상,위선자들,20171223,김충렬,시나리오(각본)
3,동창회의 목적 2|전망 좋은 집 3|외박: 친구의 여자|입영전야|개인교수: 심화학습...,개인교수: 심화학습,20252789,고도,시나리오(각본)
4,비스티걸스|바리새인,비스티걸스,20238326,김태균,제작
5,출국|원더풀 고스트|명당|인랑|마녀|7년의 밤|독전|타클라마칸|염력|1987|강철비...,원더풀 고스트,20293722,김성진,특수촬영
6,내 처제가 된 첫사랑|정사 2018|노래방 : 화끈한 여자들|유부녀들의 맛|밀애 :...,언니,20269709,김민,동시녹음


In [83]:
data = writer_code.merge(writer_detail, how='left', on=['movieNm', 'peopleCd'])
# .groupby(['movieCd', 'peopleCd']).first()
print(data.shape)
data.sample()

data = data[data['peopleNm_y'].notnull()].drop_duplicates()
# len(data['movieCd'].unique())
# len(data[data['moviePartNm'] == '감독']['movieCd'])
writer_merge = data[data['moviePartNm'] == '시나리오(각본)']
writer_merge = writer_merge.drop('peopleNm_x', axis=1)
writer_merge = writer_merge.rename(columns={'peopleNm_y':'peopleNm'})
print(writer_merge.shape)
writer_merge

(10, 9)
(4, 8)


,filmoNames,movieNm,peopleCd,repRoleNm,movieCd,moviePartNm,peopleNm,sex
1,유부녀들,유부녀들,20227564,시나리오(각본),20155879.0,시나리오(각본),신정규,남자
2,위선자들|캐치미|슈퍼맨 강보상,위선자들,20171223,시나리오(각본),20157191.0,시나리오(각본),김충렬,남자
3,동창회의 목적 2|전망 좋은 집 3|외박: 친구의 여자|입영전야|개인교수: 심화학습...,개인교수: 심화학습,20252789,시나리오(각본),20158911.0,시나리오(각본),고도,남자
4,비스티걸스|바리새인,비스티걸스,20238326,제작,20164981.0,시나리오(각본),김태균,남자


In [87]:
# merge 하면서 없어진 작가
# 태양을 쏴라 김정행
# 원더풀 고스트 김성진
# 언니 김민

writer_detail[writer_detail['peopleNm'] == '김민'] # 영화인코드 다름 - writer_code 테이블에 있는 코드로 검색해서 찾음: 남자 (김민혁)
writer_detail[writer_detail['peopleNm'] == '김정행'] # 아예 없음
writer_detail[writer_detail['peopleNm'] == '김성진'] # 영화인코드 다름

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
26,20170513,동네사람들,기획,10004631,김성진,남자
27,20170513,동네사람들,제작,10004631,김성진,남자
28,20167903,원더풀 고스트,시나리오(각본),10004631,김성진,남자
29,20167903,원더풀 고스트,기획,10004631,김성진,남자
30,20167903,원더풀 고스트,제작,10004631,김성진,남자
31,20167903,원더풀 고스트,프로듀서,10004631,김성진,남자
32,20161366,"그래, 가족",제작,10004631,김성진,남자
33,20143510,함정,제작,10004631,김성진,남자
34,20143510,함정,기획,10004631,김성진,남자
35,20124937,감기,프로듀서,10004631,김성진,남자


In [92]:
# 영화인 코드가 다른 감독들을 직접 찾아줌
writer_movieNm = ['원더풀 고스트'
                    , '언니'
]

writer_peopleNm = ['김성진'
                     , '김민']

writer_filmo = ['동네사람들|원더풀 고스트|그래, 가족|함정|레디액션 청춘|감기|그대를 사랑합니다|마을금고 연쇄습격사건|싸움의 기술|마파도|맹부삼천지교|결혼은, 미친짓이다'
                 , '언니']

writer_neg = pd.DataFrame(writer_movieNm, writer_peopleNm).reset_index().rename(columns={'index':'peopleNm', 0:'movieNm'})
writer_neg['filmoNames'] = writer_filmo
data = writer_neg.merge(writer_detail, how='left', on=['peopleNm', 'movieNm'])
writer_neg = data[data['moviePartNm'] == '시나리오(각본)']
writer_neg['sex'] = writer_neg['sex'].fillna('남자')
writer_neg

,peopleNm,movieNm,filmoNames,movieCd,moviePartNm,peopleCd,sex
0,김성진,원더풀 고스트,"동네사람들|원더풀 고스트|그래, 가족|함정|레디액션 청춘|감기|그대를 사랑합니다|마...",20167903,시나리오(각본),10004631,남자
4,김민,언니,언니,20184187,시나리오(각본),20312772,남자


In [94]:
writer_add = pd.concat([writer_merge, writer_neg], ignore_index=True)
writer_add.loc[writer_add['peopleNm'] == '김성진', 'repRoleNm'] = '제작'
writer_add.loc[writer_add['peopleNm'] == '김민', 'repRoleNm'] = '시나리오(각본)'
writer_add

,filmoNames,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,repRoleNm,sex
0,유부녀들,20155879.0,유부녀들,시나리오(각본),20227564,신정규,시나리오(각본),남자
1,위선자들|캐치미|슈퍼맨 강보상,20157191.0,위선자들,시나리오(각본),20171223,김충렬,시나리오(각본),남자
2,동창회의 목적 2|전망 좋은 집 3|외박: 친구의 여자|입영전야|개인교수: 심화학습...,20158911.0,개인교수: 심화학습,시나리오(각본),20252789,고도,시나리오(각본),남자
3,비스티걸스|바리새인,20164981.0,비스티걸스,시나리오(각본),20238326,김태균,제작,남자
4,"동네사람들|원더풀 고스트|그래, 가족|함정|레디액션 청춘|감기|그대를 사랑합니다|마...",20167903.0,원더풀 고스트,시나리오(각본),10004631,김성진,제작,남자
5,언니,20184187.0,언니,시나리오(각본),20312772,김민,시나리오(각본),남자


# 새로 추가한 감독, 작가 정보 films.csv 와 붙이기

In [97]:
df = pd.read_csv('../01.Data_Collection/films.csv')

print(df.shape)
df.sample()

(810, 38)


,순위,영화명,영화코드,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,F-rated,감독,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모
200,33.0,널 기다리며,20145396,2016-03-10,5.234303e+09,0.6%,5.234303e+09,635235.0,635235.0,551.0,33774.0,한국,한국,"(주)영화사수작,모티브픽쳐스(주),(주)디씨지플러스",(주)넥스트엔터테인먼트월드(NEW),청소년관람불가,스릴러,1,모홍진,모홍진,10026501.0,남자,감독,안시성|널 기다리며|우리 동네,모홍진,모홍진,모홍진,10026501.0,남자,감독,안시성|널 기다리며|우리 동네,"심은경,윤제문,김성오,오태경,정해균,안재홍,김원해,지대한,이호열,장준현,정재숙,정진...",심은경,희주,10040874.0,여자,배우,"신문기자|염력|특별시민|조작된 도시|걷기왕|서울역|궁합|널 기다리며|로봇, 소리|수..."


## 감독정보 채우기

In [103]:
data = df[df['대표감독'].isnull()].drop(['대표감독', '감독코드', '감독성별', '감독_대표역할', '감독필모'], axis=1)

data = data.merge(director_add, how='left', left_on='영화코드', right_on='movieCd')
data = data.drop(['movieNm', 'moviePartNm', 'movieCd'], axis=1)
data = data.rename(columns={'filmoNames': '감독필모', 'peopleCd':'감독코드', 'peopleNm':'대표감독', 'repRoleNm':'감독_대표역할', 'sex':'감독성별'})
director_add_merge = pd.concat([df[df['대표감독'].notnull()], data])

젝스키스 에이틴, 트와이스 랜드, 러브유어셀프 인 서울 제외 감독정보 모두 채움

In [110]:
director_add_merge
print(director_add_merge.shape)
director_add_merge.sample()
director_add_merge[director_add_merge['감독'].isnull()]

# 대표감독 상세는 있으나 감독 컬럼이 비어있는 데이터 채워줌
director_add_merge['감독'] = director_add_merge['감독'].fillna(director_add_merge['대표감독'])
director_add_merge[director_add_merge['영화명'] == '선종 무문관']

(810, 38)


,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
22,0,윤용진,감독,남자,20110475.0,선종 무문관|할|내 마음 속에 작은 영웅들,2018-12-13,3012.0,NaN,3012.0,25023800.0,윤용진,NaN,윤용진,윤용진,NaN,25023800.0,NaN,NaN,NaN,"우상전, 조용주, 고동업, 안홍진, 송유담, 임대혁, 최민재",배우,NaN,0.0,NaN,127.0,NaN,3.0,선종 무문관,20183144,윤용진,감독,남자,20110475.0,선종 무문관|할|내 마음 속에 작은 영웅들,NaN,NaN,우상전


## 옴니버스 영화 감독 수정

In [186]:
omni = director_add_merge.tail(10)[['영화명', '감독', '대표감독']].iloc[2:8]
omni

,영화명,감독,대표감독
16,세컨어카운트,"박수영,권칠인,권호영",권칠인
17,자각몽,"박수영,권칠인,권호영",권호영
18,과대망상자들,"최익환,이광국",신연식
19,소주와 아이스크림,"최익환,이광국",이광국
20,히치하이커,"서은아,윤재호,장은연",윤재호
21,"소년, 소녀를 만나다","서은아,윤재호,장은연",장은연


In [187]:
for i, row in omni.iterrows():
    director_add_merge.loc[director_add_merge['영화명'] == row['영화명'], '감독'] = row['대표감독']

director_add_merge.tail(10)[['영화명', '감독', '대표감독']]

,영화명,감독,대표감독
14,트와이스랜드,NaN,NaN
15,러브 유어셀프 인 서울,NaN,NaN
16,세컨어카운트,권칠인,권칠인
17,자각몽,권호영,권호영
18,과대망상자들,신연식,신연식
19,소주와 아이스크림,이광국,이광국
20,히치하이커,윤재호,윤재호
21,"소년, 소녀를 만나다",장은연,장은연
22,선종 무문관,윤용진,윤용진
23,두 번째 겨울,김의곤,김의곤


## 작가 채우기

In [188]:
df = director_add_merge
data = df[df['대표작가'].isnull()].drop(['대표작가', '작가코드', '작가성별', '작가_대표역할', '작가필모'], axis=1)

data = data.merge(writer_add, how='left', left_on='영화코드', right_on='movieCd')
data = data.drop(['movieNm', 'moviePartNm', 'movieCd'], axis=1)
data = data.rename(columns={'filmoNames': '작가필모', 'peopleCd':'작가코드', 'peopleNm':'대표작가', 'repRoleNm':'작가_대표역할', 'sex':'작가성별'})
writer_add_merge = pd.concat([df[df['대표작가'].notnull()], data])

print(writer_add_merge.shape)
writer_add_merge.sample()

(810, 38)


,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
249,3,조성은,감독,여자,20215487.0,우리 연애의 이력,2016-06-29,3043.0,한국,3043.0,24084800.0,조성은,한국,황윤경,황윤경,15세이상관람가,24084800.0,0.0%,(주)모멘텀엔터테인먼트,NaN,"전혜빈,신민철,황미영,박주영,박지영,박진영,박솔희,강 숙,양종곤,채조은,이강우,이근...",배우,여자,10061532.0,"힘을 내요, 미스터 리|인어전설|럭키|우리 연애의 이력|몽정기2|령",475.0,119.0,26.0,우리 연애의 이력,20154803,"황윤경, 조성은",시나리오(각본),여자,20256115.0,우리 연애의 이력,멜로/로맨스,더블엔비컴퍼니(주),전혜빈


In [189]:
writer_add_merge[writer_add_merge['대표작가'].isnull()]

# 대표작가 상세는 있으나 작가 컬럼이 비어있는 데이터 채워줌
writer_add_merge['작가'] = writer_add_merge['작가'].fillna(writer_add_merge['대표작가'])
writer_add_merge[writer_add_merge['영화명'] == '언니']

,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
93,1,임경택,제작,남자,10058467.0,언니|히어로|더 웹툰: 예고살인|고래를 찾는 자전거|김복남 살인사건의 전말|이브의 ...,2019-01-01,197134.0,한국,198377.0,1.757528e+09,임경택,한국,김민,김민,청소년관람불가,1.746373e+09,0.2%,"(주)제이앤씨미디어그룹,TCO(주)더콘텐츠온",NaN,"이시영,박세완,이준혁,최진호",배우,여자,10056359.0,언니|신의 한 수|더 웹툰: 예고살인|남자사용설명서|커플즈|위험한 상견례|빨간모자의...,11315.0,46.0,530.0,언니,20184187,김민,시나리오(각본),남자,20312772.0,언니,액션,"필름에이픽쳐스,(유)조이앤시네마",이시영


In [190]:
set(writer_add_merge[writer_add_merge['작가'].isnull()]['영화명']) - set(writer_add_merge[writer_add_merge['대표작가'].isnull()]['영화명'])

set()

In [191]:
writer_add_merge
print(writer_add_merge.shape)
writer_add_merge.sample()

(810, 38)


,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
80,1,전성호,감독,남자,20213586.0,이바라키의 여름,2015-08-06,568.0,한국,568.0,3711500.0,전성호,한국,권초원,권초원,전체관람가,3711500.0,0.0%,(주)씨에이씨엔터테인먼트,NaN,"김가나,김성화,김시유,김혜령,양태열,양혜정,이승언",배우,여자,20229604.0,이바라키의 여름,50.0,236.0,5.0,이바라키의 여름,20153313,권초원,시나리오(각본),NaN,20229601.0,이바라키의 여름,다큐멘터리,부산MBC,김가나


In [192]:
writer_add_merge.isnull().sum()

F-rated       0
감독            3
감독_대표역할       3
감독성별          6
감독코드          3
감독필모          3
개봉일           0
관객수           0
국적            2
누적관객수         0
누적매출액         0
대표감독          3
대표국적          2
대표작가        131
대표작가(필명)    131
등급            2
매출액           0
매출액점유율        2
배급사           3
배역명         452
배우            0
배우_대표역할       0
배우성별         18
배우코드          0
배우필모         71
상영횟수          0
순위            2
스크린수          0
영화명           0
영화코드          0
작가          131
작가_대표역할     131
작가성별        165
작가코드        131
작가필모        131
장르            2
제작사          22
주연배우          0
dtype: int64

# 누락된 작가 정보 중, 감독 = 각본 인 항목 채우기

In [193]:
data = pd.read_csv('../01.Data_Collection/writer_Null.csv', encoding='euc-kr')

writer_is_director = data[data['감독 동일인물 여부'] == '1'].drop(['Unnamed: 6', '영화인코드'], axis=1)
print(writer_is_director.shape)
writer_is_director

(32, 6)


,개봉일,영화명,감독,배우,작가,감독 동일인물 여부
0,2015.1.22,생각보다 맑은,한지원,"엄상현,양정화,한지원,한지원,이홍수,이홍수,이홍수,이호민,이영기,유아람,서정원,서정원",한지원,1
2,2015.6.24,연평해전,김학순,"김무열,진구,김율호,이현우,이완,이청아,천민희,이한종,박찬희,박혜준,최순조,최승준,...",김학순,1
7,2015.10.22,맛있는 비행,전병철,"손덕기,정유진,신유주",최상훈,1
9,2015.11.12,나도 때론 포르노그라피의 주인공이고 싶다,장성수,"최리호,김영근,서갑숙,한지은",장성수,1
16,2015.12.31,내부자들: 디 오리지널,우민호,"신지수,이병헌,조승우,백윤식,이경영,김홍파,배성우,조재윤,김대명,조우진,이엘,정만식...",우민호,1
18,2016.1.7,잡아야 산다,오인천,"김승우,김정태,한상혁,신강우,김민규,문용석,오만석,최호중,이해준,조창근,윤다영,서범...",오인천,1
23,2016.2.24,귀향,조정래,"손숙,강하나,서미지,최리,정인기,오지혜,김시은,이채현,임성철,임혜영,박주현,박지희,...",조정래,1
26,2016.4.7,"날, 보러와요",이철하,"강예원,이상윤,최진호,지대한,김종수,이학주,장태성,유건,길해연,조재윤,이형석,박송해...",이철하,1
27,2016.5.4,탐정 홍길동: 사라진 마을,조성희,"이제훈,김성균,박근형,정성화,고아라,노정의,김하나,유승목,박정원,박지훈,오현암,유정...",조성희,1
28,2016.5.26,레나,김도원,"박기림,김재만,최호중,이창배,이창배,박정민,고재철,이승민,이유리,이은빈,황지은,김근...",김도원,1


In [194]:
data[data['작가'] == 'Null']

,개봉일,영화명,감독,배우,작가,감독 동일인물 여부,Unnamed: 6,영화인코드
8,2015.10.22,오디세오,정성복,-,Null,Null,공연,NaN
10,2015.11.19,용주골,최우성,"민도윤,이채담,진서현,이서현,김민호",Null,Null,에로영화,NaN
11,2015.11.19,꾸루꾸루와 친구들: 무지개 나무의 비밀,문제대,서유리,Null,Null,뮤지컬,NaN
12,2015.11.26,서태지밴드 콘서트투어 '콰이어트나이트',서태지,서태지,Null,Null,공연,NaN
14,2015.12.3,내 친구의 아내,탁승오,"정태민,탁민",Null,Null,에로영화,NaN
15,2015.12.3,감금의 시간,김대성,"전유진,허동원,김윤재,이용주",Null,Null,에로영화,NaN
17,2016.1.7,하숙집 2,정태령,"류현아,정넘쳐,이소희",Null,Null,에로영화,NaN
19,2016.1.14,"금지된 섹스, 달콤한 복수",이숭환,"송은채,유장영",Null,Null,에로영화,NaN
20,2016.1.28,뜨거운 이웃,이전,"이채담,이유리",Null,Null,에로영화,NaN
21,2016.2.11,야한 누나들,최우성,"이채담,이미나,김라윤",Null,Null,에로영화,NaN


작가 Null 값인 데이터 대부분 에로영화, 다큐멘터리, 애니메이션, 공연, 단편영화 등

In [195]:
'연평해전' in list(writer_is_director['영화명'])

True

In [196]:
a = []
for i, row in writer_add_merge.iterrows():

    if row['영화명'] in list(writer_is_director['영화명']):
        a.append((row['영화명'], row['감독'], row['대표감독'], row['대표감독'], row['감독코드'], row['감독필모'], row['감독_대표역할'], row['감독성별']))
        
writer_is_director_writer = pd.DataFrame(a, columns=['영화명', '작가', '대표작가', '대표작가(필명)', '작가코드', '작가필모', '작가_대표역할', '작가성별'])
print(writer_is_director_writer.shape)
writer_is_director_writer.sample()

(32, 8)


,영화명,작가,대표작가,대표작가(필명),작가코드,작가필모,작가_대표역할,작가성별
8,내부자들: 디 오리지널,우민호,우민호,우민호,20111283.0,남산의 부장들|마약왕|내부자들: 디 오리지널|내부자들|간첩|파괴된 사나이|누가 예수...,감독,남자


In [203]:
data = writer_add_merge[writer_add_merge['작가'].isnull()].drop(['대표작가', '대표작가(필명)', '작가코드', '작가필모', '작가_대표역할', '작가성별'], axis=1)
print(data.shape)
data.sample()

data.merge(writer_is_director_writer, how='left', on='영화명')[['영화명', '작가_x', '작가_y']]
writer_is_director_merge = data.merge(writer_is_director_writer, how='left', on='영화명')
writer_is_director_merge = writer_is_director_merge.drop('작가_x', axis=1)
writer_is_director_merge = writer_is_director_merge.rename(columns={'작가_y': '작가'})

final_merge = pd.concat([writer_add_merge[writer_add_merge['작가'].notnull()], writer_is_director_merge])
print(final_merge.shape)
final_merge.sample()

(131, 32)
(810, 38)


,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
600,2,김인선,감독,여자,20187958.0,어른도감|수요기도회|아빠의 맛|알레르기|스테파니|김인선 감독 특별전(인디스데이)|제...,2018-08-23,6956.0,한국,6956.0,53139700.0,김인선,한국,김인선,김인선,12세이상관람가,53139700.0,0.0%,(주)영화사 진진,NaN,"엄태구,이재인,서정연,김새벽,손강국,이정은,김경옥,김민주,김기현,김수환,김연주,김윤...",배우,남자,20111842.0,판소리 복서|낙원의 밤|안시성|어른도감|가려진 시간|밀정|차이나타운|살인캠프|잉투기...,981.0,103.0,58.0,어른도감,20181821,"김인선, 박근범",감독,여자,20187958.0,어른도감|수요기도회|아빠의 맛|알레르기|스테파니|김인선 감독 특별전(인디스데이)|제...,"코미디,드라마",한국영화아카데미,엄태구


In [204]:
final_merge.isnull().sum()

F-rated       0
감독            3
감독_대표역할       3
감독성별          6
감독코드          3
감독필모          3
개봉일           0
관객수           0
국적            2
누적관객수         0
누적매출액         0
대표감독          3
대표국적          2
대표작가        103
대표작가(필명)    103
등급            2
매출액           0
매출액점유율        2
배급사           3
배역명         452
배우            0
배우_대표역할       0
배우성별         18
배우코드          0
배우필모         71
상영횟수          0
순위            2
스크린수          0
영화명           0
영화코드          0
작가          103
작가_대표역할     103
작가성별        137
작가코드        103
작가필모        103
장르            2
제작사          22
주연배우          0
dtype: int64

In [205]:
final_merge[final_merge['작가성별'].isnull() & final_merge['작가'].notnull()]

,F-rated,감독,감독_대표역할,감독성별,감독코드,감독필모,개봉일,관객수,국적,누적관객수,누적매출액,대표감독,대표국적,대표작가,대표작가(필명),등급,매출액,매출액점유율,배급사,배역명,배우,배우_대표역할,배우성별,배우코드,배우필모,상영횟수,순위,스크린수,영화명,영화코드,작가,작가_대표역할,작가성별,작가코드,작가필모,장르,제작사,주연배우
80,1,전성호,감독,남자,20213586.0,이바라키의 여름,2015-08-06,568.0,한국,568.0,3.711500e+06,전성호,한국,권초원,권초원,전체관람가,3.711500e+06,0.0%,(주)씨에이씨엔터테인먼트,NaN,"김가나,김성화,김시유,김혜령,양태열,양혜정,이승언",배우,여자,20229604.0,이바라키의 여름,50.0,236.0,5.0,이바라키의 여름,20153313,권초원,시나리오(각본),NaN,20229601.0,이바라키의 여름,다큐멘터리,부산MBC,김가나
135,0,강훈,감독,남자,20186360.0,착한 처제|야관문: 욕망의 꽃,2015-11-19,588.0,한국,588.0,3.723000e+06,강훈,한국,여필주,여필주,청소년관람불가,3.723000e+06,0.0%,쟈비스미디어,NaN,"강성필,장혜영",배우,남자,10000693.0,착한 처제|유감스러운 도시|가면|지금 사랑하는 사람과 살고 있습니까?|투사부일체|나...,82.0,233.0,19.0,착한 처제,20158385,여필주,시나리오(각본),NaN,20227240.0,착한 처제,드라마,"쟈비스미디어,PK미디어크레이티브",강성필
215,1,박흥식,감독,남자,20110989.0,"해어화|협녀, 칼의 기억|천국의 아이들|사랑해, 말순씨|인어 공주|나도 아내가 있었...",2016-04-13,485695.0,한국,485695.0,3.768201e+09,박흥식,한국,하영준,하영준,15세이상관람가,3.768201e+09,0.4%,롯데쇼핑㈜롯데엔터테인먼트,정소율,"한효주,유연석,천우희,박성웅,장영남,이한위,류혜영,김수안,차지연,장인섭,박찬미,장희...",배우,여자,10087820.0,"인랑|골든슬럼버|해어화|뷰티 인사이드|서툴지만, 사랑|쎄시봉|감시자들|반창꼬|광해,...",32322.0,35.0,572.0,해어화,20145506,하영준,시나리오(각본),NaN,20232049.0,해어화,"사극,드라마",더램프(주),한효주
224,1,손석우,감독,남자,20234506.0,안녕자두야 서유기 : 어쩌다 영웅 |극장판 안녕 자두야|안녕 자두야 스페셜 : 인어공주,2016-05-04,280406.0,한국,280406.0,2.137638e+09,손석우,한국,조민주,조민주,전체관람가,2.137638e+09,0.2%,씨제이이앤엠(주),NaN,"여민정,양정화,이수진",배우,여자,20234807.0,극장판 공룡메카드: 타이니소어의 섬|극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라...,7758.0,42.0,523.0,극장판 안녕 자두야,20162704,조민주,시나리오(각본),NaN,10065836.0,극장판 안녕 자두야|풍뎅이뎅이,애니메이션,"아툰즈,씨제이이앤엠(주)",여민정
225,0,조성희,VFX 아티스트,NaN,20200534.0,도어락|물괴|탐정 홍길동: 사라진 마을|족구왕,2016-05-04,1430885.0,한국,1430885.0,1.151617e+10,조성희,한국,조성희,조성희,15세이상관람가,1.151617e+10,1.2%,씨제이이앤엠(주),홍길동,"이제훈,김성균,박근형,정성화,고아라,노정의,김하나,유승목,박정원,박지훈,오현암,유정...",배우,남자,10057349.0,컬렉터|사냥의 시간(가제)|지구: 놀라운 하루|아이 캔 스피크|박열|탐정 홍길동: ...,41076.0,19.0,764.0,탐정 홍길동: 사라진 마을,20142406,조성희,VFX 아티스트,NaN,20200534.0,도어락|물괴|탐정 홍길동: 사라진 마을|족구왕,"드라마,액션",(주)영화사비단길,이제훈
250,0,이우철,감독,남자,10056854.0,사냥|7광구|첼로-홍미주 일가 살인사건|해피 에로 크리스마스|내츄럴시티,2016-06-29,645922.0,한국,645922.0,4.912659e+09,이우철,한국,천진우,천진우,15세이상관람가,4.912659e+09,0.5%,롯데쇼핑㈜롯데엔터테인먼트,문기성,"안성기,신재훈,오주환,조진웅,한예리,이상화,한동원,권율,손현주,박병은,예수정,차순배...",배우,남자,10043371.0,"광화문|종이꽃|사자|인간, 공간, 시간 그리고 인간|청춘 합창단 - 또 하나의 꿈|...",28862.0,32.0,696.0,사냥,20154906,천진우,시나리오(각본),NaN,20215586.0,봉오동 전투|사냥,액션,(주)빅스톤픽쳐스,안성기
255,0,변진호,감독,남자,20238366.0,빅뱅 메이드,2016-06-30,55582.0,한국,55582.0,4.664867e+08,변진호,한국,홍서율,홍서율,12세이상관람가,4.664867e+08,0.1%,씨제이 씨지브이(CJ CGV)(주),NaN,"권지용,태양,최승현,대성,승리",배우,남자,10069845.0,빅뱅 메이드|원 오브 어 카인드 3D ; G-DRAGON 2013 1ST WORLD...,1794.0,68.0,56.0,빅뱅 메이드,20165042,홍서율,시나리오(각본),NaN,20263435.0,빅뱅 메이드,다큐멘터리,(주)와이지엔터테인먼트,권지용
372,1,윤세영,감독,남자,20274889.0,마리안느와 마가렛,2017-04-20,28722.0,한국,28722.0,2.002284e+08,윤세영,한국,양희,양희,전체관람가,2.002284e+08,0.0%,(주)팝엔터테인먼트,NaN,"마리안느 스퇴거,마가렛 피사렉,박정환",배우,여자,10022075.0,마리안느와 마가렛,1271.0,73.0,64.0,마리안느와 마가렛,20172641,양희,시나리오(각본),NaN,20276950.0,마리안느와 마가렛,다큐멘터리,(주)기린제작사,마리안느 스퇴거
374,2,"홍주연,홍현정",감독,여자,20275649.0,"서서평, 천천히 평온하게",2017-04-26,123344.0,한국,123344.0,8.780904e+08,홍주연,한국,한지원,한지원,전체관람가,8.780904e+08,0.1%,커넥트픽쳐스(주),NaN,"윤안나,안은새,강다연,도세진,민주홍,박사라,이상운,진미혜,최용수,최진",배우,여자,20276709.0,"서서평, 천천히 평온하게",4402.0,58.0,234.0,"서서평, 천천히 평온하게",20173301,한지원,시나리오(각본),NaN,20276711.0,"서서평, 천천히 평온하게",다큐멘터리,재단법인 온누리선교재단(CGNTV),윤안나
390,0,임공삼,라인프로듀서,남자,10058471.0,우리들의 일기|비스티 보이즈|바보|풀리쉬 게임,2017-06-01,478.0,한국,478.0,3.496000e+06,임공삼,한국,은하수,은하수,청소년관람불가,3.496000e+06,0.0%,THE 픽쳐스,수호,"윤성모,신지훈,정한비,조민호,이형원,공정환,정대곤,박종훈,이수민,이재우,이정현,이정...",배우,남자,20196672.0,수상한 이웃|우리들의 일기|사랑만의 언어,115.0,248.0,8.0,우리들의 일기,20170941,은하수,시나리오(각본),NaN,20278929.0,우리들의 일기,"액션,드라마","(주)은하수엔터테인먼트,(주)에이디엠엔터테인먼트",윤성모
